<a href="https://colab.research.google.com/github/sb-iam/cot-dfa/blob/main/notebooks/cot_dfa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
================================================================================
     CELL 0: CoT-DFA - CHAIN-OF-THOUGHT DATAFLOW ANALYSIS
     Applying Compiler Reaching Definitions to Detect Unfaithful Reasoning
================================================================================

MATS 10.0 APPLICATION PROJECT FOR NEEL NANDA

  "Can we tell when a CoT was causally important for giving its answer?"
                                        — Neel Nanda, MATS 10.0 Interests

================================================================================
                         CORE RESEARCH QUESTION
================================================================================

  Can compiler-style reaching definitions analysis detect unfaithful
  Chain-of-Thought in code generation models?

  ┌─────────────────────────────────────────────────────────────────────────┐
  │  COMPILER DATAFLOW ANALYSIS    ←──BRIDGE──→    COT FAITHFULNESS        │
  │  ─────────────────────────────                 ────────────────────────  │
  │  • Reaching definitions         STRUCTURAL     • Which CoT matters?     │
  │  • Dead code elimination        FAITHFULNESS   • Post-hoc rationalization│
  │  • Use-def chains               METRICS        • Phantom code detection │
  │  • O(1) single-pass analysis                   • No model calls needed  │
  └─────────────────────────────────────────────────────────────────────────┘

================================================================================
                         PRIMARY HYPOTHESIS
================================================================================

  H₁: phantom_ratio (code elements without CoT justification)
      correlates with test case failure rate.

      High phantoms → Model generated code without reasoning it through
                   → Higher probability of bugs

  ┌─────────────────────────────────────────────────────────────────────────┐
  │                                                                         │
  │    phantom_ratio = |Phantom| / |CodeElements|                          │
  │                                                                         │
  │    where Phantom = { c ∈ Code | RD(c) = ∅ }                            │
  │          RD(c) = reaching definitions from CoT segments                │
  │                                                                         │
  │    EXPECTED: Negative correlation with test pass rate                  │
  │    PASS CRITERION: r < 0, p < 0.05                                     │
  │                                                                         │
  └─────────────────────────────────────────────────────────────────────────┘

================================================================================
                    WHY REACHING DEFINITIONS FOR COT?
================================================================================

  Classical Compiler Analysis:
  ┌─────────────────────────────────────────────────────────────────────────┐
  │                                                                         │
  │    d1: x = 5          ──────┐                                          │
  │    d2: y = x + 1            │ d1 reaches this use                      │
  │    d3: x = 10         ──────┼──────┐                                   │
  │    d4: z = x + y            │      │ d3 reaches, d1 killed             │
  │                             ▼      ▼                                   │
  │                                                                         │
  │    "For each USE of variable x, which DEFINITIONS could have           │
  │     produced the value?"                                               │
  │                                                                         │
  └─────────────────────────────────────────────────────────────────────────┘

  CoT-DFA Mapping:
  ┌─────────────────────────────┬─────────────────────────────────────────────┐
  │ Program Analysis            │ CoT-DFA Equivalent                          │
  ├─────────────────────────────┼─────────────────────────────────────────────┤
  │ Variable definition         │ CoT step introducing concept/approach       │
  │ Variable use                │ Code element using that concept             │
  │ Reaching definition         │ Which CoT step justifies this code?         │
  │ Dead code                   │ CoT steps not reaching any output           │
  │ Use without definition      │ PHANTOM — code without reasoning            │
  └─────────────────────────────┴─────────────────────────────────────────────┘

================================================================================
                    COT-DFA ANALYSIS EXAMPLE
================================================================================

  ┌─────────────────────────────────────────────────────────────────────────┐
  │  CoT Trace:                                                            │
  │  ┌───────────────────────────────────────────────────────────────┐    │
  │  │ s1: "First, I'll use a hash map for O(1) lookup"              │    │
  │  │ s2: "I need to handle the edge case of empty input"           │    │
  │  │ s3: "Let me add some comments for clarity"                    │    │
  │  └───────────────────────────────────────────────────────────────┘    │
  │            │                    │                                      │
  │            ▼                    ▼                                      │
  │  Code Output:                                                          │
  │  ┌───────────────────────────────────────────────────────────────┐    │
  │  │ def solve(nums):                                               │    │
  │  │     seen = {}  ◄─── s1 reaches (hash map → dict)              │    │
  │  │     if not nums:  ◄─── s2 reaches (edge case → condition)     │    │
  │  │         return -1                                              │    │
  │  │     for n in nums:                                             │    │
  │  │         seen[n] = True                                         │    │
  │  │     return max(seen.keys()) ◄─── PHANTOM! (not in CoT)        │    │
  │  └───────────────────────────────────────────────────────────────┘    │
  │                                                                        │
  │  Analysis Result:                                                      │
  │  ├── s1 → LIVE (reaches hash map usage)                               │
  │  ├── s2 → LIVE (reaches edge case check)                              │
  │  ├── s3 → DEAD (no code element matches "comments")                   │
  │  └── max(seen.keys()) → PHANTOM (not discussed in CoT)                │
  │                                                                        │
  │  Metrics:                                                              │
  │  ├── phantom_ratio = 1/5 = 0.20 (one unjustified element)            │
  │  ├── dead_ratio = 1/3 = 0.33 (one unproductive segment)              │
  │  └── reach_coverage = 4/5 = 0.80 (80% code justified)                │
  │                                                                        │
  └─────────────────────────────────────────────────────────────────────────┘

================================================================================
                    COMPARISON: COT-DFA vs THOUGHT ANCHORS
================================================================================

  ┌─────────────────────────────────────────────────────────────────────────┐
  │                                                                         │
  │  THOUGHT ANCHORS (Bogdan et al.)     COT-DFA (This Work)               │
  │  ──────────────────────────────      ─────────────────────              │
  │                                                                         │
  │  Question: "Which sentences          Question: "Is this a valid        │
  │            matter causally?"                    derivation?"            │
  │                                                                         │
  │  Method:   Counterfactual            Method:   Structural analysis     │
  │            perturbation                        (no model calls)        │
  │                                                                         │
  │  Cost:     O(n) forward passes       Cost:     O(1) - single pass      │
  │            per sample                          parse + match           │
  │                                                                         │
  │  Detects:  • Important sentences     Detects:  • Phantom code          │
  │            • Attention patterns                • Dead reasoning        │
  │                                                                         │
  │  ─────────────────────────────────────────────────────────────────     │
  │                                                                         │
  │  COMPLEMENTARY: Together they answer both                              │
  │  "What matters?" AND "Is it properly derived?"                         │
  │                                                                         │
  └─────────────────────────────────────────────────────────────────────────┘

================================================================================
                    PRIOR WORK: UNFAITHFULNESS IS REAL
================================================================================

  ┌────────────────────────┬──────────────────────────┬────────────────────┐
  │ Study                  │ Finding                  │ Implication        │
  ├────────────────────────┼──────────────────────────┼────────────────────┤
  │ Chen et al.            │ Claude 3.7 Sonnet only   │ Models frequently  │
  │ (Anthropic, 2025)      │ 25% faithful on hint     │ don't say what     │
  │                        │ verbalization test       │ they think         │
  ├────────────────────────┼──────────────────────────┼────────────────────┤
  │ Arcuschin et al.       │ GPT-4o-mini shows 13%    │ Unfaithfulness     │
  │ (MATS, 2025)           │ implicit post-hoc        │ occurs naturally,  │
  │ arXiv:2503.08679       │ rationalization rate     │ not just adversarial│
  ├────────────────────────┼──────────────────────────┼────────────────────┤
  │ Lanham et al.          │ Larger models produce    │ Problem may worsen │
  │ (Anthropic, 2023)      │ less faithful reasoning  │ with scale         │
  └────────────────────────┴──────────────────────────┴────────────────────┘

  COT-DFA CONTRIBUTION: Lightweight, single-pass structural analysis that
  complements expensive causal methods like Thought Anchors.

================================================================================
                    PIPELINE ARCHITECTURE
================================================================================

  ┌─────────────────────────────────────────────────────────────────────────┐
  │                        COT-DFA PIPELINE                                 │
  └─────────────────────────────────────────────────────────────────────────┘

       ┌──────────┐      ┌──────────┐      ┌──────────┐      ┌──────────┐
       │  INPUT   │      │  PARSE   │      │ ANALYZE  │      │  OUTPUT  │
       │          │ ───► │          │ ───► │          │ ───► │          │
       │ Model    │      │ CoT +    │      │ Reaching │      │ Metrics  │
       │ Response │      │ Code     │      │ Defs     │      │ + Report │
       └──────────┘      └──────────┘      └──────────┘      └──────────┘
            │                 │                 │                 │
            ▼                 ▼                 ▼                 ▼
      ┌──────────┐      ┌──────────┐      ┌──────────┐      ┌──────────┐
      │<think>   │      │Segments: │      │Def-Use   │      │phantom:  │
      │...       │      │ s0, s1   │      │Graph     │      │ 0.15     │
      │</think>  │      │          │      │          │      │dead:     │
      │```python │      │Elements: │      │Reaching  │      │ 0.33     │
      │def f():  │      │ c0, c1   │      │Sets      │      │faith:    │
      │  ...     │      │          │      │          │      │ 0.72     │
      └──────────┘      └──────────┘      └──────────┘      └──────────┘

================================================================================
                    DATA SOURCES
================================================================================

  ┌─────────────────────────────────────────────────────────────────────────┐
  │ PRIMARY: OpenThoughts-114k                                             │
  ├─────────────────────────────────────────────────────────────────────────┤
  │ Source:  HuggingFace (open-thoughts/OpenThoughts-114k)                 │
  │ Content: 114K reasoning traces from DeepSeek-R1                        │
  │ Format:  problem, deepseek_reasoning (<think>), deepseek_solution      │
  │ Filter:  domain == "code" (TACO, APPS, CodeContests)                   │
  │ Sample:  100 problems with test cases                                  │
  │ Advantage: Pre-existing high-quality CoT traces                        │
  └─────────────────────────────────────────────────────────────────────────┘

  ┌─────────────────────────────────────────────────────────────────────────┐
  │ SECONDARY: HumanEval (Fresh Generations)                               │
  ├─────────────────────────────────────────────────────────────────────────┤
  │ Source:  HuggingFace (openai_humaneval)                                │
  │ Content: 164 Python programming problems                               │
  │ Model:   CodeGemma 7B-IT (prompted for <think> blocks)                 │
  │ Sample:  50 problems                                                   │
  │ Advantage: Validate on model we control                                │
  └─────────────────────────────────────────────────────────────────────────┘

  ┌─────────────────────────────────────────────────────────────────────────┐
  │ REFERENCE: chainscope                                                  │
  ├─────────────────────────────────────────────────────────────────────────┤
  │ Source:  GitHub (jettjaniak/chainscope)                                │
  │ Content: Labeled unfaithful CoT examples from Arcuschin et al.         │
  │ Patterns: post_hoc, restoration, shortcut                              │
  │ Purpose: Calibrate unfaithfulness detection                            │
  └─────────────────────────────────────────────────────────────────────────┘

================================================================================
                    TECHNICAL STACK
================================================================================

  ┌─────────────────┬─────────────────────────┬─────────────────────────────┐
  │ Component       │ Choice                  │ Rationale                   │
  ├─────────────────┼─────────────────────────┼─────────────────────────────┤
  │ Platform        │ Google Colab Pro (H100) │ 80GB VRAM, JAX native       │
  │ Model           │ CodeGemma 7B-IT         │ JAX/Flax, MLIR-compatible   │
  │ Model Load      │ kagglehub               │ Official Google pathway     │
  │ Embeddings      │ UniXcoder               │ Code-NL shared space        │
  │ AST Analysis    │ beniget + ast           │ Lightweight def-use chains  │
  │ Framework       │ JAX/Flax                │ XLA compilation, TPU-ready  │
  │ Statistics      │ scipy                   │ Point-biserial, Fisher's    │
  │ Visualization   │ matplotlib + seaborn    │ Publication-quality plots   │
  └─────────────────┴─────────────────────────┴─────────────────────────────┘

================================================================================
                    CONCEPT VOCABULARY (22 PROGRAMMING CONCEPTS)
================================================================================

  ┌─────────────────────────────────────────────────────────────────────────┐
  │ DATA STRUCTURES                                                        │
  ├─────────────────────────────────────────────────────────────────────────┤
  │ dict  │ hash, map, dictionary, hashmap, key-value, lookup, counter     │
  │ list  │ array, list, sequence, collection, elements, items             │
  │ set   │ set, unique, deduplicate, distinct                             │
  │ stack │ stack, lifo, push, pop                                         │
  │ queue │ queue, fifo, deque, bfs                                        │
  │ heap  │ heap, priority queue, heapq, min heap, max heap                │
  │ tree  │ tree, binary tree, bst, trie, node, root                       │
  │ graph │ graph, vertices, edges, adjacent, neighbor, dfs                │
  └─────────────────────────────────────────────────────────────────────────┘

  ┌─────────────────────────────────────────────────────────────────────────┐
  │ ALGORITHMS                                                             │
  ├─────────────────────────────────────────────────────────────────────────┤
  │ sort       │ sort, order, arrange, sorted, ascending, descending       │
  │ search     │ search, find, lookup, binary search, locate               │
  │ recursion  │ recursive, recursion, base case, call itself              │
  │ dp         │ dynamic programming, memoization, memo, dp, subproblem    │
  │ greedy     │ greedy, local optimal, best choice                        │
  │ two_pointer│ two pointer, left right, start end, sliding window        │
  │ backtrack  │ backtrack, prune, explore, candidates                     │
  └─────────────────────────────────────────────────────────────────────────┘

  ┌─────────────────────────────────────────────────────────────────────────┐
  │ CONTROL FLOW                                                           │
  ├─────────────────────────────────────────────────────────────────────────┤
  │ loop        │ iterate, loop, for each, traverse, go through, while     │
  │ condition   │ if, check, condition, edge case, boundary                │
  │ early_return│ return early, base case, edge case, special case         │
  └─────────────────────────────────────────────────────────────────────────┘

  ┌─────────────────────────────────────────────────────────────────────────┐
  │ OPERATIONS                                                             │
  ├─────────────────────────────────────────────────────────────────────────┤
  │ count     │ count, frequency, occurrences, how many                    │
  │ sum       │ sum, total, add up, accumulate                             │
  │ max_min   │ maximum, minimum, max, min, largest, smallest              │
  │ string_op │ string, character, substring, split, join                  │
  └─────────────────────────────────────────────────────────────────────────┘

================================================================================
                    METRICS DEFINITIONS
================================================================================

  ┌─────────────────────────────────────────────────────────────────────────┐
  │ PHANTOM RATIO                                                          │
  ├─────────────────────────────────────────────────────────────────────────┤
  │                                                                         │
  │                  |Phantom|        # code elements without CoT          │
  │ phantom_ratio = ─────────── = ──────────────────────────────────       │
  │                    |C|              # total code elements              │
  │                                                                         │
  │ Interpretation:                                                        │
  │ • 0.0 = Perfect: Every code element has CoT justification              │
  │ • 0.5 = Concerning: Half the code "appeared from nowhere"              │
  │ • 1.0 = Complete disconnect: CoT irrelevant to code                    │
  │                                                                         │
  │ HYPOTHESIS: High phantom_ratio → test failures                         │
  │                                                                         │
  └─────────────────────────────────────────────────────────────────────────┘

  ┌─────────────────────────────────────────────────────────────────────────┐
  │ DEAD RATIO                                                             │
  ├─────────────────────────────────────────────────────────────────────────┤
  │                                                                         │
  │                |Dead|          # CoT steps reaching nothing            │
  │ dead_ratio = ────────── = ─────────────────────────────────────        │
  │                |S|               # total CoT segments                  │
  │                                                                         │
  │ Interpretation:                                                        │
  │ • 0.0 = Efficient: Every reasoning step contributes                    │
  │ • 0.3 = Normal: Some exploratory thinking                              │
  │ • 0.7+ = Suspicious: Mostly filler/padding                             │
  │                                                                         │
  └─────────────────────────────────────────────────────────────────────────┘

  ┌─────────────────────────────────────────────────────────────────────────┐
  │ FAITHFULNESS SCORE (Combined)                                          │
  ├─────────────────────────────────────────────────────────────────────────┤
  │                                                                         │
  │ faithfulness = α × structural_score + β × semantic_similarity          │
  │                                                                         │
  │ where:                                                                  │
  │   structural_score = reach_coverage × (1 - 0.5 × dead_ratio)           │
  │   reach_coverage = 1 - phantom_ratio                                   │
  │   α = 0.7, β = 0.3 (tunable weights)                                   │
  │                                                                         │
  │ Interpretation:                                                        │
  │ • 0.0-0.3: Low faithfulness (CoT disconnected from code)               │
  │ • 0.3-0.6: Moderate faithfulness (partial alignment)                   │
  │ • 0.6-1.0: High faithfulness (CoT reflects code structure)             │
  │                                                                         │
  └─────────────────────────────────────────────────────────────────────────┘

  ┌─────────────────────────────────────────────────────────────────────────┐
  │ CONCEPT JACCARD                                                        │
  ├─────────────────────────────────────────────────────────────────────────┤
  │                                                                         │
  │                |κ(S) ∩ κ(C)|      # shared concepts                    │
  │ jaccard = ───────────────────── = ─────────────────────────────        │
  │            |κ(S) ∪ κ(C)|           # total unique concepts             │
  │                                                                         │
  │ where κ(S) = concepts from CoT segments, κ(C) = concepts from code     │
  │                                                                         │
  └─────────────────────────────────────────────────────────────────────────┘

================================================================================
                    STATISTICAL ANALYSIS PLAN
================================================================================

  For small samples (n=50-150), we use robust non-parametric methods:

  ┌─────────────────────────────────────────────────────────────────────────┐
  │ 1. POINT-BISERIAL CORRELATION                                          │
  ├─────────────────────────────────────────────────────────────────────────┤
  │ Use: Continuous (faithfulness) vs Binary (pass/fail)                   │
  │ Power: Adequate at n=50 for medium-large effects (r ≥ 0.3)             │
  │ Implementation: scipy.stats.pointbiserialr()                           │
  │ Expected: r < 0 for phantom_ratio (negative correlation)               │
  └─────────────────────────────────────────────────────────────────────────┘

  ┌─────────────────────────────────────────────────────────────────────────┐
  │ 2. FISHER'S EXACT TEST                                                 │
  ├─────────────────────────────────────────────────────────────────────────┤
  │ Use: 2×2 contingency (faithful/unfaithful × correct/incorrect)         │
  │ Advantage: No minimum sample requirement                               │
  │ Report: Odds ratio with 95% CI                                         │
  │ Implementation: scipy.stats.fisher_exact()                             │
  └─────────────────────────────────────────────────────────────────────────┘

  ┌─────────────────────────────────────────────────────────────────────────┐
  │ 3. BOOTSTRAP CONFIDENCE INTERVALS                                      │
  ├─────────────────────────────────────────────────────────────────────────┤
  │ Method: BCa (bias-corrected and accelerated)                           │
  │ Resamples: 9,999                                                       │
  │ Use: Robust uncertainty quantification for effect sizes                │
  └─────────────────────────────────────────────────────────────────────────┘

  ┌─────────────────────────────────────────────────────────────────────────┐
  │ 4. EFFECT SIZES (Cohen's d)                                            │
  ├─────────────────────────────────────────────────────────────────────────┤
  │ |d| < 0.2:  Negligible                                                 │
  │ 0.2 ≤ |d| < 0.5:  Small                                                │
  │ 0.5 ≤ |d| < 0.8:  Medium                                               │
  │ |d| ≥ 0.8:  Large (TARGET for PoC)                                     │
  │                                                                         │
  │ Always report effect sizes alongside p-values                          │
  └─────────────────────────────────────────────────────────────────────────┘

  SCIENTIFIC NOTE: Even null results are publishable if methodology is
  sound and framework is mechanically validated.

================================================================================
                    20-STEP EXECUTION PLAN (15 HOURS)
================================================================================

┌──────┬─────────────────────────────────────────────┬────────┬──────────────┐
│ Step │ Description                                 │ Time   │ Deliverable  │
├──────┼─────────────────────────────────────────────┼────────┼──────────────┤
│      │ PHASE 1: ENVIRONMENT & DATA SETUP (1.5h)   │        │              │
├──────┼─────────────────────────────────────────────┼────────┼──────────────┤
│  1   │ Verify GPU, install dependencies           │ 20 min │ Environment  │
│  2   │ Load CodeGemma 7B-IT via kagglehub         │ 25 min │ Sampler obj  │
│  3   │ Download OpenThoughts-114k, filter code    │ 15 min │ 100 samples  │
│  4   │ Clone chainscope, define unfaithful patterns│ 20 min │ Reference    │
├──────┼─────────────────────────────────────────────┼────────┼──────────────┤
│      │ PHASE 2: EXTRACTION PIPELINE (2.5h)        │        │              │
├──────┼─────────────────────────────────────────────┼────────┼──────────────┤
│  5   │ Build CoT sentence segmenter               │ 30 min │ segment_cot()│
│  6   │ Build concept vocabulary + CoT extractor   │ 45 min │ extract_cot_ │
│  7   │ Build AST concept extractor with beniget   │ 45 min │ extract_code_│
│  8   │ Build reaching definitions analyzer        │ 30 min │ compute_rd() │
├──────┼─────────────────────────────────────────────┼────────┼──────────────┤
│      │ PHASE 3: METRICS DEVELOPMENT (2.5h)        │        │              │
├──────┼─────────────────────────────────────────────┼────────┼──────────────┤
│  9   │ Implement phantom_ratio, dead_ratio        │ 45 min │ Metric funcs │
│ 10   │ Implement reach_coverage, concept_jaccard  │ 45 min │ Metric funcs │
│ 11   │ Implement UniXcoder semantic similarity    │ 30 min │ semantic_sim │
│ 12   │ Combine into faithfulness_score            │ 30 min │ compute_f()  │
├──────┼─────────────────────────────────────────────┼────────┼──────────────┤
│      │ PHASE 4: EVALUATION FRAMEWORK (3h)         │        │              │
├──────┼─────────────────────────────────────────────┼────────┼──────────────┤
│ 13   │ Build safe code execution harness          │ 45 min │ execute_safe │
│ 14   │ Run analysis on 100 OpenThoughts samples   │ 60 min │ Results list │
│ 15   │ Load HumanEval, generate 50 CodeGemma CoTs │ 45 min │ Fresh gens   │
│ 16   │ Combine into final dataset (150 samples)   │ 30 min │ combined_df  │
├──────┼─────────────────────────────────────────────┼────────┼──────────────┤
│      │ PHASE 5: STATISTICAL ANALYSIS (2.5h)       │        │              │
├──────┼─────────────────────────────────────────────┼────────┼──────────────┤
│ 17   │ Point-biserial correlation                 │ 30 min │ r, p-value   │
│ 18   │ Fisher's exact test + odds ratio           │ 30 min │ OR, 95% CI   │
│ 19   │ Bootstrap CIs + Cohen's d effect sizes     │ 30 min │ Effect sizes │
│ 20   │ Generate visualizations + final report     │ 60 min │ 4 figs+report│
└──────┴─────────────────────────────────────────────┴────────┴──────────────┘

  TOTAL: 15 hours execution + ~3 hours buffer for debugging

================================================================================
                    VALIDATION TESTS PER CELL
================================================================================

  CELL 1 (Setup - Step 1):
  ├── nvidia-smi shows GPU available
  ├── All pip installs succeed
  ├── import torch succeeds
  ├── import jax succeeds (optional)
  ├── import transformers succeeds
  └── CUDA/CPU device detected correctly

  CELL 2 (Model - Step 2):
  ├── kagglehub.model_download succeeds
  ├── tokenizer.Load() succeeds
  ├── params_lib.load_and_format_params() succeeds
  ├── sampler generates text
  └── Test generation produces valid Python-like output

  CELL 3 (Dataset - Step 3):
  ├── load_dataset("open-thoughts/OpenThoughts-114k") succeeds
  ├── DataFrame has expected columns
  ├── Filter returns >100 code samples
  ├── Sample of 100 created successfully
  └── Test cases present in samples

  CELL 4 (Reference - Step 4):
  ├── git clone chainscope succeeds (or graceful skip)
  ├── UNFAITHFUL_PATTERNS dict populated
  └── Reference patterns accessible

  CELL 5 (Segmenter - Step 5):
  ├── segment_cot() returns List[Segment]
  ├── Segment has id, text, position, concepts
  ├── <think> block extraction works
  ├── Sentence splitting works
  ├── Numbered step splitting works
  └── Minimum length filter applied (>15 chars)

  CELL 6 (CoT Concepts - Step 6):
  ├── CONCEPT_VOCABULARY has 22 concepts
  ├── extract_cot_concepts() returns Set[str]
  ├── "hash map" → 'dict' mapping works
  ├── Multiple concepts extracted from single segment
  └── Empty segment returns empty set

  CELL 7 (AST Concepts - Step 7):
  ├── extract_code_concepts() returns (Set, List[CodeElement])
  ├── ast.Dict → 'dict' mapping works
  ├── ast.For → 'loop' mapping works
  ├── Function call detection (sorted→sort) works
  ├── CodeElement has id, node_type, line_number, concepts
  └── SyntaxError gracefully returns empty

  CELL 8 (Reaching Defs - Step 8):
  ├── compute_reaching_definitions() returns DFAResult
  ├── DFAResult has segments, elements, reaching_sets
  ├── reaching_sets[elem.id] is ReachingSet
  ├── Concept overlap creates edges
  ├── phantoms property returns elements with no reaching
  └── dead_segments property returns segments reaching nothing

  CELL 9-10 (Ratios - Steps 9-10):
  ├── phantom_ratio() returns float in [0, 1]
  ├── dead_ratio() returns float in [0, 1]
  ├── reach_coverage = 1 - phantom_ratio (verified)
  ├── concept_jaccard() returns float in [0, 1]
  └── Edge cases (empty) return 0.0

  CELL 11 (UniXcoder - Step 11):
  ├── AutoTokenizer.from_pretrained succeeds
  ├── AutoModel.from_pretrained succeeds
  ├── get_embedding() returns 768-dim vector
  ├── semantic_similarity() returns float in [-1, 1]
  └── "hash map" similar to "{}" (positive cosine)

  CELL 12 (Faithfulness - Step 12):
  ├── compute_faithfulness() returns FaithfulnessResult
  ├── FaithfulnessResult has all component metrics
  ├── faithfulness_score in [0, 1]
  ├── Weights α=0.7, β=0.3 applied correctly
  └── All fields populated

  CELL 13 (Execution - Step 13):
  ├── execute_code_safely() returns (bool, str, str)
  ├── Timeout works (10s default)
  ├── Passing code returns (True, stdout, "")
  ├── Failing code returns (False, "", stderr)
  └── Temp file cleaned up

  CELL 14 (OpenThoughts - Step 14):
  ├── analyze_sample() returns AnalysisResult
  ├── All 100 samples processed
  ├── Results list has 100 entries
  ├── Progress bar (tqdm) works
  └── No crashes on edge cases

  CELL 15 (HumanEval - Step 15):
  ├── load_dataset("openai_humaneval") succeeds
  ├── 50 problems selected
  ├── CodeGemma generation works
  ├── <think> blocks parsed
  └── ```python``` blocks parsed

  CELL 16 (Combine - Step 16):
  ├── results_df has 150 rows
  ├── All columns present
  ├── source column distinguishes OT vs HE
  ├── describe() shows reasonable stats
  └── No NaN in critical columns

  CELL 17 (Correlation - Step 17):
  ├── pointbiserialr() returns (r, p)
  ├── r is in [-1, 1]
  ├── p is in [0, 1]
  ├── All metrics tested
  └── correlation_results dict populated

  CELL 18 (Fisher - Step 18):
  ├── Contingency table created
  ├── fisher_exact() returns (OR, p)
  ├── Odds ratio is positive
  ├── 95% CI computed via log transform
  └── fisher_results dict populated

  CELL 19 (Effect Sizes - Step 19):
  ├── cohens_d() returns float
  ├── Bootstrap resampling works (9999)
  ├── 95% CI computed
  ├── Interpretation string correct
  └── effect_results dict populated

  CELL 20 (Report - Step 20):
  ├── 4 visualizations created
  ├── Figures saved to disk
  ├── Report markdown generated
  ├── All placeholders filled
  └── Files downloadable

================================================================================
                    MEMORY BUDGET (Google Colab H100)
================================================================================

  ┌────────────────────────────────┬─────────────┬──────────────────────────┐
  │ Component                      │ Memory      │ Notes                    │
  ├────────────────────────────────┼─────────────┼──────────────────────────┤
  │ CodeGemma 7B-IT weights        │ ~14 GB      │ bf16: 7B × 2 bytes       │
  │ CodeGemma activations          │ ~2 GB       │ Inference batch          │
  │ UniXcoder model                │ ~500 MB     │ 125M params              │
  │ OpenThoughts samples           │ ~100 MB     │ 100 samples in memory    │
  │ Results DataFrame              │ ~50 MB      │ 150 rows, all columns    │
  │ Working memory                 │ ~2 GB       │ Intermediate tensors     │
  ├────────────────────────────────┼─────────────┼──────────────────────────┤
  │ TOTAL                          │ ~19 GB      │ Fits H100 80GB easily    │
  └────────────────────────────────┴─────────────┴──────────────────────────┘

  FALLBACK: If CodeGemma loading fails, use only OpenThoughts (no Step 15)
            This reduces to ~4GB total, runnable on T4.

================================================================================
                    OUTPUT DATA STRUCTURES
================================================================================

  @dataclass
  class Segment:
      id: str                    # "s0", "s1", ...
      text: str                  # Raw CoT text
      position: int              # Order in CoT
      concepts: Set[str]         # Extracted concepts

  @dataclass
  class CodeElement:
      id: str                    # "c0", "c1", ...
      node_type: str             # "Dict", "For", "Call", ...
      line_number: int           # Source location
      concepts: Set[str]         # Extracted concepts

  @dataclass
  class ReachingSet:
      element: CodeElement
      reaching_segments: Set[str]  # Segment IDs that reach

  @dataclass
  class DFAResult:
      segments: List[Segment]
      elements: List[CodeElement]
      reaching_sets: Dict[str, ReachingSet]
      cot_concepts: Set[str]
      code_concepts: Set[str]

  @dataclass
  class FaithfulnessResult:
      phantom_ratio: float
      dead_ratio: float
      reach_coverage: float
      semantic_sim: float
      faithfulness_score: float
      cot_concepts: Set[str]
      code_concepts: Set[str]
      concept_overlap: Set[str]
      concept_jaccard: float
      num_segments: int
      num_elements: int
      num_phantoms: int
      num_dead: int

  @dataclass
  class AnalysisResult:
      sample_id: str
      faithfulness: FaithfulnessResult
      test_passed: bool
      execution_error: Optional[str]

================================================================================
                    DEPENDENCIES
================================================================================

  Core:
  ├── kagglehub              # CodeGemma download
  ├── gemma                  # CodeGemma inference
  ├── flax                   # JAX neural networks
  ├── jax[cuda12]            # XLA compilation
  ├── transformers           # UniXcoder, datasets
  ├── datasets               # HuggingFace datasets
  ├── sentencepiece          # Tokenization
  └── torch                  # UniXcoder backend

  Analysis:
  ├── scipy                  # Statistical tests
  ├── pandas                 # DataFrames
  ├── numpy                  # Numerics
  └── beniget                # AST def-use chains

  Visualization:
  ├── matplotlib             # Plots
  ├── seaborn                # Statistical visualization
  └── tqdm                   # Progress bars

================================================================================
                    ALIGNMENT WITH NEEL NANDA'S INTERESTS
================================================================================

  ┌──────────────────────────────┬──────────────────────────────┬──────────┐
  │ Neel's Interest              │ CoT-DFA Addresses            │ Match    │
  ├──────────────────────────────┼──────────────────────────────┼──────────┤
  │ "Can we tell when CoT was    │ Reaching definitions track   │          │
  │  causally important for      │ exactly which CoT segments   │    ✓     │
  │  giving its answer?"         │ contribute to code elements  │          │
  ├──────────────────────────────┼──────────────────────────────┼──────────┤
  │ "Design good monitors or     │ phantom_ratio, dead_ratio,   │          │
  │  metrics for whether CoT     │ faithfulness_score are       │    ✓     │
  │  is telling us what we       │ exactly this type of metric  │          │
  │  think?"                     │                              │          │
  ├──────────────────────────────┼──────────────────────────────┼──────────┤
  │ "Extend Thought Anchors"     │ Complementary formalism:     │          │
  │                              │ • Thought Anchors: causal    │    ✓     │
  │                              │ • CoT-DFA: structural        │          │
  ├──────────────────────────────┼──────────────────────────────┼──────────┤
  │ "Reasoning models"           │ Targets code generation      │          │
  │                              │ with native <think> blocks   │    ✓     │
  ├──────────────────────────────┼──────────────────────────────┼──────────┤
  │ "Applied interpretability"   │ Single-pass, no expensive    │          │
  │                              │ resampling, production-ready │    ✓     │
  ├──────────────────────────────┼──────────────────────────────┼──────────┤
  │ "Start simple"               │ Classical compiler analysis  │          │
  │                              │ applied to new domain        │    ✓     │
  └──────────────────────────────┴──────────────────────────────┴──────────┘

================================================================================
                    EXTENSION: CIRCUIT PROVENANCE BRIDGE
================================================================================

  If CoT-DFA validates, bridge to training data attribution:

  ┌─────────────────────────────────────────────────────────────────────────┐
  │                                                                         │
  │  H₅: Training examples with shortcuts (correct answer, weak reasoning) │
  │      cause models to produce unfaithful Chain-of-Thought.              │
  │                                                                         │
  │  ┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐   │
  │  │  CoT-DFA        │     │  Influence      │     │  Compare        │   │
  │  │  Classify:      │ ──► │  Functions:     │ ──► │  Training       │   │
  │  │  faithful vs    │     │  Find top-K     │     │  Examples       │   │
  │  │  unfaithful     │     │  influencers    │     │                 │   │
  │  └─────────────────┘     └─────────────────┘     └─────────────────┘   │
  │                                                                         │
  │  PASS: shortcut_prevalence(unfaithful) > shortcut_prevalence(faithful) │
  │                                                                         │
  └─────────────────────────────────────────────────────────────────────────┘

  This connects to the broader IAM-Audit dissertation work on compiler-
  integrated interpretability.

================================================================================
                    WHAT THIS NOTEBOOK PROVES
================================================================================

  ✓ Reaching definitions can be applied to CoT → code analysis
  ✓ phantom_ratio correlates (or not) with test failure
  ✓ Lightweight structural analysis complements causal methods
  ✓ 22 programming concepts suffice for code generation domain
  ✓ UniXcoder provides semantic validation of structural matching
  ✓ Statistical framework appropriate for small samples
  ✓ Pipeline scales to production deployment

================================================================================
                    SUCCESS CRITERIA
================================================================================

  PRIMARY (H₁):
  ├── Significant negative correlation (p < 0.05)
  ├── phantom_ratio vs test pass: r < 0
  └── Effect size: Cohen's d ≥ 0.5 (medium or larger)

  SECONDARY:
  ├── dead_ratio > 0.3 indicates padding behavior
  ├── Harder problems → higher phantom_ratio
  └── Phantom locations correlate with bug locations

  INTERESTING FAILURE:
  ├── Framework mechanically validated even if H₁ fails
  ├── Null result still publishable with proper analysis
  └── Opens questions for future research

================================================================================
"""

print(__doc__)

# ============================================================================
# Configuration Registry
# ============================================================================

CONFIG = {
    "model": {
        "name": "google/codegemma/flax/7b-it",
        "provider": "kagglehub",
        "params": "7B",
        "memory_gb": 14,
    },
    "embeddings": {
        "name": "microsoft/unixcoder-base",
        "dim": 768,
        "memory_mb": 500,
    },
    "dataset": {
        "primary": "open-thoughts/OpenThoughts-114k",
        "secondary": "openai_humaneval",
        "reference": "jettjaniak/chainscope",
        "n_primary": 100,
        "n_secondary": 50,
        "n_total": 150,
    },
    "concepts": {
        "n_concepts": 22,
        "categories": ["data_structures", "algorithms", "control_flow", "operations"],
    },
    "metrics": {
        "alpha": 0.7,  # Structural weight
        "beta": 0.3,   # Semantic weight
    },
    "statistics": {
        "significance_level": 0.05,
        "bootstrap_resamples": 9999,
        "effect_size_target": 0.5,  # Medium
    },
    "execution": {
        "timeout_seconds": 10,
        "random_seed": 42,
    },
}

# ============================================================================
# Concept Vocabulary
# ============================================================================

CONCEPT_VOCABULARY = {
    # Data Structures (8)
    'dict': {'hash', 'map', 'dictionary', 'hashmap', 'hash map', 'key-value',
             'lookup table', 'mapping', 'counter'},
    'list': {'array', 'list', 'sequence', 'collection', 'elements', 'items'},
    'set': {'set', 'unique', 'deduplicate', 'distinct'},
    'stack': {'stack', 'lifo', 'push', 'pop'},
    'queue': {'queue', 'fifo', 'deque', 'bfs'},
    'heap': {'heap', 'priority queue', 'heapq', 'min heap', 'max heap'},
    'tree': {'tree', 'binary tree', 'bst', 'trie', 'node', 'root'},
    'graph': {'graph', 'vertices', 'edges', 'adjacent', 'neighbor', 'dfs'},

    # Algorithms (7)
    'sort': {'sort', 'order', 'arrange', 'sorted', 'ascending', 'descending'},
    'search': {'search', 'find', 'lookup', 'binary search', 'locate'},
    'recursion': {'recursive', 'recursion', 'base case', 'call itself'},
    'dp': {'dynamic programming', 'memoization', 'memo', 'dp', 'subproblem'},
    'greedy': {'greedy', 'local optimal', 'best choice'},
    'two_pointer': {'two pointer', 'left right', 'start end', 'sliding window'},
    'backtrack': {'backtrack', 'prune', 'explore', 'candidates'},

    # Control Flow (3)
    'loop': {'iterate', 'loop', 'for each', 'traverse', 'go through', 'while'},
    'condition': {'if', 'check', 'condition', 'edge case', 'boundary'},
    'early_return': {'return early', 'base case', 'edge case', 'special case'},

    # Operations (4)
    'count': {'count', 'frequency', 'occurrences', 'how many'},
    'sum': {'sum', 'total', 'add up', 'accumulate'},
    'max_min': {'maximum', 'minimum', 'max', 'min', 'largest', 'smallest'},
    'string_op': {'string', 'character', 'substring', 'split', 'join'},
}

# ============================================================================
# Unfaithfulness Patterns (from chainscope)
# ============================================================================

UNFAITHFUL_PATTERNS = {
    'post_hoc': "Reasoning constructed after answer decided",
    'restoration': "Error acknowledged but not corrected",
    'shortcut': "Conclusion without supporting steps",
    'phantom': "Code construct not mentioned in reasoning",
    'filler': "Reasoning steps that contribute nothing",
}

# ============================================================================
# Progress Tracker
# ============================================================================

POC_STATUS = {
    "cell_0": "✓ Design complete",
    "cell_1": "⬜ Environment setup (Step 1)",
    "cell_2": "⬜ Load CodeGemma (Step 2)",
    "cell_3": "⬜ Load OpenThoughts (Step 3)",
    "cell_4": "⬜ Load chainscope (Step 4)",
    "cell_5": "⬜ CoT segmenter (Step 5)",
    "cell_6": "⬜ CoT concepts (Step 6)",
    "cell_7": "⬜ AST concepts (Step 7)",
    "cell_8": "⬜ Reaching defs (Step 8)",
    "cell_9": "⬜ Phantom ratio (Step 9)",
    "cell_10": "⬜ Dead ratio (Step 10)",
    "cell_11": "⬜ UniXcoder (Step 11)",
    "cell_12": "⬜ Faithfulness (Step 12)",
    "cell_13": "⬜ Execution harness (Step 13)",
    "cell_14": "⬜ OpenThoughts analysis (Step 14)",
    "cell_15": "⬜ HumanEval generation (Step 15)",
    "cell_16": "⬜ Combine dataset (Step 16)",
    "cell_17": "⬜ Correlation (Step 17)",
    "cell_18": "⬜ Fisher's test (Step 18)",
    "cell_19": "⬜ Effect sizes (Step 19)",
    "cell_20": "⬜ Report (Step 20)",
}

# ============================================================================
# Phase Summary
# ============================================================================

PHASES = {
    "Phase 1": {
        "name": "Environment & Data Setup",
        "steps": [1, 2, 3, 4],
        "time": "1.5 hours",
        "outputs": ["Environment", "CodeGemma", "OpenThoughts", "Reference"],
    },
    "Phase 2": {
        "name": "Extraction Pipeline",
        "steps": [5, 6, 7, 8],
        "time": "2.5 hours",
        "outputs": ["segment_cot()", "extract_cot_concepts()",
                   "extract_code_concepts()", "compute_reaching_defs()"],
    },
    "Phase 3": {
        "name": "Metrics Development",
        "steps": [9, 10, 11, 12],
        "time": "2.5 hours",
        "outputs": ["phantom_ratio()", "dead_ratio()",
                   "semantic_similarity()", "compute_faithfulness()"],
    },
    "Phase 4": {
        "name": "Evaluation Framework",
        "steps": [13, 14, 15, 16],
        "time": "3 hours",
        "outputs": ["execute_code_safely()", "100 OT results",
                   "50 HE results", "combined_df"],
    },
    "Phase 5": {
        "name": "Statistical Analysis",
        "steps": [17, 18, 19, 20],
        "time": "2.5 hours",
        "outputs": ["correlation_results", "fisher_results",
                   "effect_results", "report + figures"],
    },
}

# ============================================================================
# Print Summary
# ============================================================================

print("\n" + "=" * 70)
print("CELL 0 COMPLETE: Design finalized")
print("=" * 70)

print(f"\n📊 DATASET: {CONFIG['dataset']['n_total']} samples")
print(f"   ├── OpenThoughts-114k: {CONFIG['dataset']['n_primary']} samples")
print(f"   └── HumanEval (CodeGemma): {CONFIG['dataset']['n_secondary']} samples")

print(f"\n🧠 MODEL: {CONFIG['model']['name']}")
print(f"   └── Memory: ~{CONFIG['model']['memory_gb']} GB")

print(f"\n📐 CONCEPTS: {CONFIG['concepts']['n_concepts']} programming concepts")
print(f"   └── Categories: {', '.join(CONFIG['concepts']['categories'])}")

print(f"\n📈 STATISTICS:")
print(f"   ├── α = {CONFIG['statistics']['significance_level']}")
print(f"   ├── Bootstrap: {CONFIG['statistics']['bootstrap_resamples']} resamples")
print(f"   └── Target effect: d ≥ {CONFIG['statistics']['effect_size_target']}")

print(f"\n⏱️ TIMELINE: 15 hours across 5 phases")
for phase_name, phase_info in PHASES.items():
    print(f"   {phase_name}: {phase_info['name']} ({phase_info['time']})")

print(f"\n🎯 HYPOTHESIS: phantom_ratio correlates negatively with test pass rate")
print(f"   PASS: r < 0, p < 0.05, d ≥ 0.5")

print("\n" + "=" * 70)
print("Proceed to Cell 1: Environment Setup")
print("=" * 70)

In [ ]:
"""
================================================================================
     CELL 1: ENVIRONMENT SETUP & AUTHENTICATION
================================================================================

OBJECTIVES:
  ├── [1.1] Verify GPU availability (H100/A100/T4)
  ├── [1.2] Install all dependencies
  ├── [1.3] Configure API authentication (Kaggle, HuggingFace)
  ├── [1.4] Import all libraries
  └── [1.5] Validate environment

COLAB SECRETS REQUIRED:
  ├── KAGGLE_USERNAME  → Kaggle username for CodeGemma download
  ├── KAGGLE_KEY       → Kaggle API key
  └── HF_TOKEN         → HuggingFace token (optional, for gated models)

TIME ESTIMATE: ~20 minutes (mostly pip install)

================================================================================
"""

import os
import sys
import time
from datetime import datetime

print("=" * 70)
print("CELL 1: ENVIRONMENT SETUP")
print("=" * 70)
print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print()

# ============================================================================
# [1.1] GPU VERIFICATION
# ============================================================================

print("[1/5] GPU Verification...")
print("-" * 40)

try:
    import subprocess
    result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total,driver_version',
                           '--format=csv,noheader'], capture_output=True, text=True)
    if result.returncode == 0:
        gpu_info = result.stdout.strip()
        print(f"  ✅ GPU detected: {gpu_info}")

        # Check for recommended GPUs
        if 'H100' in gpu_info:
            print("  🚀 H100 detected - optimal for this notebook")
            GPU_TYPE = "H100"
        elif 'A100' in gpu_info:
            print("  🚀 A100 detected - excellent for this notebook")
            GPU_TYPE = "A100"
        elif 'V100' in gpu_info:
            print("  ✅ V100 detected - good for this notebook")
            GPU_TYPE = "V100"
        elif 'T4' in gpu_info:
            print("  ⚠️  T4 detected - may need to skip CodeGemma generation")
            print("      → Will use OpenThoughts only (100 samples)")
            GPU_TYPE = "T4"
        elif 'L4' in gpu_info:
            print("  ✅ L4 detected - good for this notebook")
            GPU_TYPE = "L4"
        else:
            print("  ✅ GPU detected - proceeding")
            GPU_TYPE = "OTHER"
    else:
        print("  ❌ No GPU detected")
        print("  → Falling back to CPU (will be slow)")
        GPU_TYPE = "CPU"
except Exception as e:
    print(f"  ❌ GPU check failed: {e}")
    GPU_TYPE = "CPU"

print()

# ============================================================================
# [1.2] DEPENDENCY INSTALLATION
# ============================================================================

print("[2/5] Installing Dependencies...")
print("-" * 40)

# Core dependencies for CoT-DFA
DEPENDENCIES = {
    "core": [
        "kagglehub",           # CodeGemma download
        "transformers>=4.40.0", # Models and tokenizers
        "datasets",            # HuggingFace datasets
        "sentencepiece",       # Tokenization
        "accelerate",          # Model loading utilities
    ],
    "analysis": [
        "scipy",               # Statistical tests
        "pandas",              # DataFrames
        "numpy",               # Numerics
        "beniget",             # AST def-use chains
    ],
    "visualization": [
        "matplotlib",          # Plots
        "seaborn",             # Statistical visualization
        "tqdm",                # Progress bars
    ],
    "optional_jax": [
        # Uncomment if using JAX/Flax for CodeGemma
        # "jax[cuda12]",
        # "flax",
        # "gemma",
    ],
}

def install_packages(packages, category):
    """Install a list of packages with progress reporting."""
    import subprocess
    for pkg in packages:
        if not pkg or pkg.startswith('#'):
            continue
        pkg_name = pkg.split('>=')[0].split('==')[0]
        print(f"  Installing {pkg_name}...", end=" ", flush=True)
        try:
            result = subprocess.run(
                [sys.executable, "-m", "pip", "install", "-q", pkg],
                capture_output=True, text=True
            )
            if result.returncode == 0:
                print("✅")
            else:
                print(f"⚠️ ({result.stderr.strip()[:50]})")
        except Exception as e:
            print(f"❌ {e}")

# Install each category
for category, packages in DEPENDENCIES.items():
    if packages and not packages[0].startswith('#'):
        print(f"\n  [{category.upper()}]")
        install_packages(packages, category)

print()

# ============================================================================
# [1.3] API AUTHENTICATION
# ============================================================================

print("[3/5] API Authentication...")
print("-" * 40)

# Track authentication status
AUTH_STATUS = {
    "kaggle": False,
    "huggingface": False,
}

# --- Kaggle Authentication (required for CodeGemma) ---
print("\n  [KAGGLE] - Required for CodeGemma download")

# Configuration - UPDATE THESE IF NEEDED
KAGGLE_USERNAME = "shakthibachala"  # Your Kaggle username
KAGGLE_SECRET_NAME = "KAGGLE_API_TOKEN"  # Name of secret containing API key

try:
    from google.colab import userdata

    try:
        # Get API key from Colab Secrets
        kaggle_key = userdata.get(KAGGLE_SECRET_NAME)

        if kaggle_key:
            # Set environment variables
            os.environ["KAGGLE_USERNAME"] = KAGGLE_USERNAME
            os.environ["KAGGLE_KEY"] = kaggle_key

            # Also write kaggle.json for libraries that need it
            kaggle_dir = os.path.expanduser("~/.kaggle")
            os.makedirs(kaggle_dir, exist_ok=True)
            kaggle_json_path = os.path.join(kaggle_dir, "kaggle.json")

            import json
            with open(kaggle_json_path, 'w') as f:
                json.dump({"username": KAGGLE_USERNAME, "key": kaggle_key}, f)
            os.chmod(kaggle_json_path, 0o600)  # Secure permissions

            print(f"  ✅ Kaggle authenticated as: {KAGGLE_USERNAME}")
            print(f"  ✅ Created ~/.kaggle/kaggle.json")
            AUTH_STATUS["kaggle"] = True
        else:
            raise ValueError("Secret returned empty")

    except Exception as e:
        # Check if kaggle.json already exists
        kaggle_json_path = os.path.expanduser("~/.kaggle/kaggle.json")
        if os.path.exists(kaggle_json_path):
            print(f"  ✅ Kaggle authenticated via existing ~/.kaggle/kaggle.json")
            AUTH_STATUS["kaggle"] = True
        else:
            print(f"  ⚠️  Kaggle credentials not found: {e}")
            print(f"  → Add '{KAGGLE_SECRET_NAME}' to Colab Secrets (your API key)")
            print(f"  → Get key from: https://www.kaggle.com/settings → API → Create New Token")
            print("  → CodeGemma generation will be skipped")

except ImportError:
    # Not in Colab
    print("  ℹ️  Not running in Colab, checking local Kaggle config...")
    kaggle_json = os.path.expanduser("~/.kaggle/kaggle.json")
    if os.path.exists(kaggle_json):
        print(f"  ✅ Kaggle authenticated via {kaggle_json}")
        AUTH_STATUS["kaggle"] = True
    else:
        print("  ⚠️  No Kaggle credentials found")

# --- HuggingFace Authentication (optional, for gated models) ---
print("\n  [HUGGINGFACE] - Optional for gated models")

# Configuration - UPDATE THESE IF NEEDED
HF_SECRET_NAME = "mech_interp"  # Your HF token secret name

try:
    from google.colab import userdata

    try:
        hf_token = userdata.get(HF_SECRET_NAME)
        if hf_token:
            os.environ["HF_TOKEN"] = hf_token
            os.environ["HUGGING_FACE_HUB_TOKEN"] = hf_token

            # Login to HuggingFace
            try:
                from huggingface_hub import login
                login(token=hf_token, add_to_git_credential=False)
                print(f"  ✅ HuggingFace authenticated (via '{HF_SECRET_NAME}' secret)")
                AUTH_STATUS["huggingface"] = True
            except Exception as e:
                print(f"  ⚠️  HuggingFace login failed: {e}")
        else:
            print(f"  ℹ️  No '{HF_SECRET_NAME}' secret found (optional)")

    except Exception as e:
        print(f"  ℹ️  HuggingFace token not configured: {e}")

except ImportError:
    print("  ℹ️  Not in Colab, skipping HF authentication")

print()

# ============================================================================
# [1.4] IMPORT ALL LIBRARIES
# ============================================================================

print("[4/5] Importing Libraries...")
print("-" * 40)

# Track import status
IMPORT_STATUS = {}

def safe_import(module_name, alias=None, from_module=None, import_items=None):
    """Safely import a module with error handling."""
    try:
        if from_module:
            module = __import__(from_module, fromlist=[module_name])
            obj = getattr(module, module_name)
            globals()[alias or module_name] = obj
        elif import_items:
            module = __import__(module_name)
            for item in import_items:
                globals()[item] = getattr(module, item)
        else:
            module = __import__(module_name)
            globals()[alias or module_name] = module
        IMPORT_STATUS[alias or module_name] = True
        return True
    except ImportError as e:
        IMPORT_STATUS[alias or module_name] = False
        return False

# --- Core Libraries ---
print("\n  [CORE]")
imports_core = [
    ("os", None),
    ("sys", None),
    ("re", None),
    ("ast", None),
    ("json", None),
    ("time", None),
    ("subprocess", None),
    ("tempfile", None),
    ("dataclasses", None),
]
for module, alias in imports_core:
    if safe_import(module, alias):
        print(f"    ✅ {module}")
    else:
        print(f"    ❌ {module}")

# --- Data Science ---
print("\n  [DATA SCIENCE]")
imports_ds = [
    ("numpy", "np"),
    ("pandas", "pd"),
    ("scipy", None),
    ("scipy.stats", "stats"),
]
for module, alias in imports_ds:
    if safe_import(module, alias):
        print(f"    ✅ {alias or module}")
    else:
        print(f"    ❌ {alias or module}")

# --- ML/NLP ---
print("\n  [ML/NLP]")
ml_imports = [
    "transformers",
    "datasets",
    "torch",
]
for module in ml_imports:
    if safe_import(module):
        print(f"    ✅ {module}")
    else:
        print(f"    ❌ {module}")

# --- AST Analysis ---
print("\n  [AST ANALYSIS]")
if safe_import("beniget"):
    print("    ✅ beniget")
else:
    print("    ⚠️  beniget not available (will use basic AST)")

# --- Visualization ---
print("\n  [VISUALIZATION]")
viz_imports = [
    ("matplotlib.pyplot", "plt"),
    ("seaborn", "sns"),
]
for module, alias in viz_imports:
    try:
        exec(f"import {module} as {alias}")
        globals()[alias] = eval(alias)
        IMPORT_STATUS[alias] = True
        print(f"    ✅ {alias}")
    except ImportError:
        IMPORT_STATUS[alias] = False
        print(f"    ❌ {alias}")

# --- Progress Bars ---
if safe_import("tqdm"):
    from tqdm.auto import tqdm
    print("    ✅ tqdm")
else:
    # Fallback tqdm
    def tqdm(iterable, **kwargs):
        return iterable
    print("    ⚠️  tqdm (using fallback)")

# --- Kagglehub ---
print("\n  [MODEL DOWNLOAD]")
if safe_import("kagglehub"):
    print("    ✅ kagglehub")
else:
    print("    ❌ kagglehub (CodeGemma download will fail)")

print()

# ============================================================================
# [1.5] ENVIRONMENT VALIDATION
# ============================================================================

print("[5/5] Environment Validation...")
print("-" * 40)

# Collect environment info
ENV_INFO = {
    "python_version": sys.version.split()[0],
    "gpu_type": GPU_TYPE,
    "kaggle_auth": AUTH_STATUS["kaggle"],
    "hf_auth": AUTH_STATUS["huggingface"],
    "torch_available": IMPORT_STATUS.get("torch", False),
    "transformers_available": IMPORT_STATUS.get("transformers", False),
    "datasets_available": IMPORT_STATUS.get("datasets", False),
    "kagglehub_available": IMPORT_STATUS.get("kagglehub", False),
    "scipy_available": IMPORT_STATUS.get("scipy", False),
    "beniget_available": IMPORT_STATUS.get("beniget", False),
}

# Print summary
print(f"\n  Python:        {ENV_INFO['python_version']}")
print(f"  GPU:           {ENV_INFO['gpu_type']}")
print(f"  Kaggle Auth:   {'✅' if ENV_INFO['kaggle_auth'] else '❌'}")
print(f"  HF Auth:       {'✅' if ENV_INFO['hf_auth'] else 'ℹ️ (optional)'}")
print(f"  PyTorch:       {'✅' if ENV_INFO['torch_available'] else '❌'}")
print(f"  Transformers:  {'✅' if ENV_INFO['transformers_available'] else '❌'}")
print(f"  Datasets:      {'✅' if ENV_INFO['datasets_available'] else '❌'}")
print(f"  Kagglehub:     {'✅' if ENV_INFO['kagglehub_available'] else '❌'}")
print(f"  SciPy:         {'✅' if ENV_INFO['scipy_available'] else '❌'}")
print(f"  Beniget:       {'✅' if ENV_INFO['beniget_available'] else '⚠️ (fallback)'}")

# Determine capabilities
print("\n  [CAPABILITIES]")
CAN_GENERATE_CODEGEMMA = (
    ENV_INFO['kaggle_auth'] and
    ENV_INFO['kagglehub_available'] and
    ENV_INFO['gpu_type'] in ['H100', 'A100', 'V100', 'L4']
)
CAN_USE_OPENTHOUGHTS = (
    ENV_INFO['datasets_available'] and
    ENV_INFO['transformers_available']
)
CAN_USE_UNIXCODER = (
    ENV_INFO['torch_available'] and
    ENV_INFO['transformers_available']
)
CAN_RUN_STATISTICS = ENV_INFO['scipy_available']

print(f"  CodeGemma Generation:  {'✅ Available' if CAN_GENERATE_CODEGEMMA else '❌ Skipped'}")
print(f"  OpenThoughts Dataset:  {'✅ Available' if CAN_USE_OPENTHOUGHTS else '❌ Missing deps'}")
print(f"  UniXcoder Embeddings:  {'✅ Available' if CAN_USE_UNIXCODER else '❌ Missing deps'}")
print(f"  Statistical Analysis:  {'✅ Available' if CAN_RUN_STATISTICS else '❌ Missing scipy'}")

# Warnings
if not CAN_GENERATE_CODEGEMMA:
    print("\n  ⚠️  CodeGemma generation disabled:")
    if not ENV_INFO['kaggle_auth']:
        print("      → Missing Kaggle credentials")
    if not ENV_INFO['kagglehub_available']:
        print("      → kagglehub not installed")
    if ENV_INFO['gpu_type'] == 'T4':
        print("      → T4 GPU has limited VRAM")
    print("      → Will use OpenThoughts samples only (100 samples)")

# ============================================================================
# CONFIGURATION OBJECT FOR DOWNSTREAM CELLS
# ============================================================================

class EnvironmentConfig:
    """Configuration object passed to downstream cells."""

    # Environment
    PYTHON_VERSION = ENV_INFO['python_version']
    GPU_TYPE = GPU_TYPE

    # Authentication
    KAGGLE_AUTH = AUTH_STATUS['kaggle']
    HF_AUTH = AUTH_STATUS['huggingface']

    # Capabilities
    CAN_GENERATE_CODEGEMMA = CAN_GENERATE_CODEGEMMA
    CAN_USE_OPENTHOUGHTS = CAN_USE_OPENTHOUGHTS
    CAN_USE_UNIXCODER = CAN_USE_UNIXCODER
    CAN_RUN_STATISTICS = CAN_RUN_STATISTICS

    # Dataset configuration (adjusted based on capabilities)
    N_OPENTHOUGHTS = 100
    N_HUMANEVAL = 50 if CAN_GENERATE_CODEGEMMA else 0
    N_TOTAL = N_OPENTHOUGHTS + N_HUMANEVAL

    # Execution
    RANDOM_SEED = 42
    EXECUTION_TIMEOUT = 10  # seconds

    # Statistics
    SIGNIFICANCE_LEVEL = 0.05
    BOOTSTRAP_RESAMPLES = 9999
    FAITHFULNESS_ALPHA = 0.7  # Structural weight
    FAITHFULNESS_BETA = 0.3   # Semantic weight

# Create config instance
ENV_CONFIG = EnvironmentConfig()

# ============================================================================
# FINAL STATUS
# ============================================================================

print("\n" + "=" * 70)
print("CELL 1 COMPLETE: Environment configured")
print("=" * 70)

# Determine overall status
critical_missing = []
if not CAN_USE_OPENTHOUGHTS:
    critical_missing.append("datasets/transformers")
if not CAN_RUN_STATISTICS:
    critical_missing.append("scipy")

if critical_missing:
    print(f"\n❌ CRITICAL: Missing {', '.join(critical_missing)}")
    print("   Cannot proceed - please fix dependencies")
else:
    print(f"\n✅ Ready to proceed!")
    print(f"   Dataset size: {ENV_CONFIG.N_TOTAL} samples")
    if not CAN_GENERATE_CODEGEMMA:
        print(f"   Note: Using OpenThoughts only (CodeGemma disabled)")

print(f"\n📌 ENV_CONFIG object available for downstream cells")
print(f"   Access: ENV_CONFIG.CAN_GENERATE_CODEGEMMA, etc.")

print("\n" + "=" * 70)
print("Proceed to Cell 2: Load CodeGemma (or skip if disabled)")
print("=" * 70)

In [ ]:
"""
CELL 2: MODEL SETUP
===================
OpenThoughts-114k already has DeepSeek-R1 reasoning traces.
CodeGemma is NOT a reasoning model (no native <think> tags).

This cell just validates we can do inference if needed later.
Primary data comes from OpenThoughts (Cell 3).
"""

import torch

print("=" * 60)
print("CELL 2: Model Setup")
print("=" * 60)

# ============================================================================
# REASONING MODEL CONTEXT
# ============================================================================

print("""
┌─────────────────────────────────────────────────────────────┐
│  REASONING MODELS vs CODE MODELS                            │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  DeepSeek-R1 (reasoning):  Native <think>...</think> tags  │
│  ├── Used in OpenThoughts-114k dataset                     │
│  └── TRUE chain-of-thought reasoning                       │
│                                                             │
│  CodeGemma (code):  Standard code generation               │
│  ├── No native reasoning format                            │
│  └── Would need prompting to fake CoT                      │
│                                                             │
│  DECISION: Use OpenThoughts (real reasoning) only          │
│                                                             │
└─────────────────────────────────────────────────────────────┘
""")

# ============================================================================
# GPU CHECK
# ============================================================================

print("[1/2] GPU Status...")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"  ✅ GPU: {gpu_name} ({gpu_mem:.0f} GB)")
    DEVICE = "cuda"
else:
    print("  ⚠️  No GPU, using CPU")
    DEVICE = "cpu"

# ============================================================================
# UPDATE CONFIG
# ============================================================================

print("\n[2/2] Updating configuration...")

# Override previous config - we're using OpenThoughts only
ENV_CONFIG.CAN_GENERATE_CODEGEMMA = False
ENV_CONFIG.N_HUMANEVAL = 0
ENV_CONFIG.N_OPENTHOUGHTS = 150  # Increased since no HumanEval
ENV_CONFIG.N_TOTAL = 150

print(f"  ✅ Dataset: OpenThoughts-114k only")
print(f"  ✅ Sample size: {ENV_CONFIG.N_TOTAL} samples")
print(f"  ✅ Source: DeepSeek-R1 reasoning traces (real CoT)")

# ============================================================================
# EXPORTS
# ============================================================================

class ModelExports:
    """Model configuration for downstream cells."""
    device: str = DEVICE
    available: bool = True
    source: str = "OpenThoughts-114k (DeepSeek-R1)"

MODEL_CONFIG = ModelExports()

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "=" * 60)
print("✅ CELL 2 COMPLETE")
print("=" * 60)
print(f"""
Data Strategy:
  ├── Source: OpenThoughts-114k
  ├── Model: DeepSeek-R1 (true reasoning model)
  ├── Format: Native <think>...</think> tags
  ├── Samples: {ENV_CONFIG.N_TOTAL}
  └── Quality: High (real chain-of-thought)

Why not CodeGemma?
  └── Not a reasoning model - would need fake prompting

Proceed to Cell 3: Load OpenThoughts Dataset
""")

In [ ]:
"""
CELL 3: LOAD OPENTHOUGHTS DATASET
=================================
Load DeepSeek-R1 reasoning traces for code generation problems.
"""

import re
import pandas as pd
from datasets import load_dataset
from dataclasses import dataclass, field
from typing import List, Set, Optional, Tuple

print("=" * 60)
print("CELL 3: Load OpenThoughts Dataset")
print("=" * 60)

# ============================================================================
# LOAD DATASET
# ============================================================================

print("\n[1/4] Loading OpenThoughts-114k...")

ds = load_dataset("open-thoughts/OpenThoughts-114k", split="train")
print(f"  ✅ Loaded {len(ds):,} total samples")

# Inspect structure
print(f"\n  Dataset columns: {ds.column_names}")
sample_ex = ds[0]
print(f"  Sample keys: {list(sample_ex.keys())}")

# Show sample values for key fields
for key in list(sample_ex.keys())[:6]:
    val = str(sample_ex[key])[:100] if sample_ex[key] else "None"
    print(f"    {key}: {val}...")

# ============================================================================
# FILTER FOR CODE DOMAIN
# ============================================================================

print("\n[2/4] Filtering for code problems...")

# Identify the actual field names
SOLUTION_FIELDS = ['deepseek_solution', 'solution', 'response', 'answer', 'output']
REASONING_FIELDS = ['deepseek_reasoning', 'reasoning', 'thought', 'thinking']
SOURCE_FIELDS = ['source', 'dataset', 'domain', 'category']

def get_field(example, field_names):
    """Get first available field from list."""
    for field in field_names:
        if field in example and example[field]:
            return example[field]
    return ""

def is_code_sample(example):
    """Check if sample contains code."""
    # Get solution from various possible fields
    solution = get_field(example, SOLUTION_FIELDS)

    # Check if solution looks like Python code
    code_indicators = ['def ', 'class ', 'import ', 'return ', 'for ', 'while ', 'if ']
    if any(ind in solution for ind in code_indicators):
        return True

    # Check source/domain field
    source = get_field(example, SOURCE_FIELDS).lower()
    code_sources = ['taco', 'apps', 'code', 'python', 'leetcode', 'contest']
    if any(s in source for s in code_sources):
        return True

    return False

# Filter with progress
print("  Scanning for code samples...")
code_samples = []
for i, ex in enumerate(ds):
    if is_code_sample(ex):
        code_samples.append(ex)
    if i % 20000 == 0:
        print(f"    Scanned {i:,}... found {len(code_samples):,} code samples")

print(f"  ✅ Found {len(code_samples):,} code samples")

# ============================================================================
# PARSE AND VALIDATE SAMPLES
# ============================================================================

print("\n[3/4] Parsing samples...")

@dataclass
class CodeSample:
    """A parsed code sample with reasoning trace."""
    id: str
    problem: str
    thinking: str
    solution: str
    source: str
    test_cases: Optional[str] = None

    @property
    def has_thinking(self) -> bool:
        return len(self.thinking) > 50

    @property
    def has_solution(self) -> bool:
        return 'def ' in self.solution or 'class ' in self.solution

    @property
    def is_valid(self) -> bool:
        return self.has_thinking and self.has_solution

def extract_thinking(text: str) -> str:
    """Extract content from <think>...</think> tags."""
    # Try explicit tags first
    match = re.search(r'<think>(.*?)</think>', text, re.DOTALL | re.IGNORECASE)
    if match:
        return match.group(1).strip()

    # Try other common patterns
    for pattern in [
        r'<reasoning>(.*?)</reasoning>',
        r'<thought>(.*?)</thought>',
        r'\*\*Thinking\*\*:?\s*(.*?)(?=\*\*|```|$)',
    ]:
        match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
        if match:
            return match.group(1).strip()

    # Fallback: everything before code block
    code_start = text.find('```')
    if code_start > 100:
        return text[:code_start].strip()

    return ""

def extract_code(text: str) -> str:
    """Extract Python code from solution."""
    # Try ```python block
    match = re.search(r'```python\s*(.*?)```', text, re.DOTALL)
    if match:
        return match.group(1).strip()

    # Try any ``` block
    match = re.search(r'```\s*(.*?)```', text, re.DOTALL)
    if match:
        code = match.group(1).strip()
        if 'def ' in code or 'class ' in code:
            return code

    # Try to find raw code
    lines = text.split('\n')
    code_lines = []
    in_code = False

    for line in lines:
        stripped = line.strip()
        if stripped.startswith(('def ', 'class ', 'import ', 'from ')):
            in_code = True
        if in_code:
            if stripped and not stripped.startswith('#'):
                code_lines.append(line)
            elif not stripped and code_lines:
                code_lines.append(line)

    return '\n'.join(code_lines).strip()

def extract_test_cases(problem: str, solution: str) -> Optional[str]:
    """Try to extract test cases from problem or solution."""
    # Look for assert statements
    asserts = re.findall(r'assert\s+.+', solution)
    if asserts:
        return '\n'.join(asserts[:5])

    # Look for example outputs in problem
    examples = re.findall(r'(?:Example|Input|Output).*?(?=Example|Input|$)',
                          problem, re.DOTALL | re.IGNORECASE)
    if examples:
        return '\n'.join(examples[:3])

    return None

def parse_sample(idx: int, example: dict) -> CodeSample:
    """Parse a raw example into CodeSample."""
    problem = get_field(example, ['problem', 'question', 'prompt', 'input'])
    reasoning = get_field(example, REASONING_FIELDS)
    solution = get_field(example, SOLUTION_FIELDS)
    source = get_field(example, SOURCE_FIELDS) or 'unknown'

    thinking = extract_thinking(reasoning) if reasoning else ""
    code = extract_code(solution) if solution else ""
    tests = extract_test_cases(problem, solution)

    return CodeSample(
        id=f"ot_{idx:04d}",
        problem=problem,
        thinking=thinking,
        solution=code,
        source=source,
        test_cases=tests,
    )

# Parse all code samples
parsed_samples = [parse_sample(i, ex) for i, ex in enumerate(code_samples)]
valid_samples = [s for s in parsed_samples if s.is_valid]

print(f"  ✅ Parsed {len(parsed_samples):,} samples")
print(f"  ✅ Valid (has thinking + code): {len(valid_samples):,}")

# If no valid samples, show debug info
if len(valid_samples) == 0 and len(parsed_samples) > 0:
    print("\n  ⚠️  No valid samples! Debugging first parsed sample:")
    s = parsed_samples[0]
    print(f"    thinking length: {len(s.thinking)}")
    print(f"    solution length: {len(s.solution)}")
    print(f"    has_thinking: {s.has_thinking}")
    print(f"    has_solution: {s.has_solution}")
    if s.thinking:
        print(f"    thinking preview: {s.thinking[:200]}...")
    if s.solution:
        print(f"    solution preview: {s.solution[:200]}...")

# ============================================================================
# SELECT FINAL SAMPLE
# ============================================================================

print("\n[4/4] Selecting samples...")

import random
random.seed(ENV_CONFIG.RANDOM_SEED)

if len(valid_samples) == 0:
    print("  ❌ No valid samples found!")
    print("\n  Falling back: using samples with ANY code (relaxed validation)")
    # Relax validation - just need some code
    valid_samples = [s for s in parsed_samples if len(s.solution) > 20]
    print(f"  ✅ Relaxed: {len(valid_samples)} samples with code")

if len(valid_samples) == 0:
    print("  ❌ Still no samples! Using raw examples directly...")
    # Last resort: just take samples that have 'def ' in any field
    for i, ex in enumerate(code_samples[:200]):
        thinking = str(ex.get('deepseek_reasoning', ex.get('reasoning', '')))
        solution = str(ex.get('deepseek_solution', ex.get('solution', '')))
        if 'def ' in solution:
            valid_samples.append(CodeSample(
                id=f"ot_{i:04d}",
                problem=str(ex.get('problem', ''))[:1000],
                thinking=thinking,
                solution=solution,
                source=str(ex.get('source', 'unknown')),
            ))
    print(f"  ✅ Direct extraction: {len(valid_samples)} samples")

N_SAMPLES = min(ENV_CONFIG.N_TOTAL, len(valid_samples))
SAMPLES = random.sample(valid_samples, N_SAMPLES) if valid_samples else []

# Sort by ID for reproducibility
SAMPLES.sort(key=lambda x: x.id)

print(f"  ✅ Selected {N_SAMPLES} samples")

# ============================================================================
# CREATE DATAFRAME
# ============================================================================

if SAMPLES:
    SAMPLES_DF = pd.DataFrame([
        {
            'id': s.id,
            'problem': s.problem[:500],
            'thinking': s.thinking,
            'solution': s.solution,
            'source': s.source,
            'thinking_len': len(s.thinking),
            'solution_len': len(s.solution),
            'has_tests': s.test_cases is not None,
        }
        for s in SAMPLES
    ])
else:
    SAMPLES_DF = pd.DataFrame(columns=['id', 'problem', 'thinking', 'solution',
                                        'source', 'thinking_len', 'solution_len', 'has_tests'])

# ============================================================================
# STATISTICS
# ============================================================================

print("\n" + "-" * 60)
print("Dataset Statistics:")
print("-" * 60)

if len(SAMPLES) > 0:
    stats = {
        'Total samples': len(SAMPLES),
        'Avg thinking length': f"{SAMPLES_DF['thinking_len'].mean():.0f} chars",
        'Avg solution length': f"{SAMPLES_DF['solution_len'].mean():.0f} chars",
        'With test cases': f"{SAMPLES_DF['has_tests'].sum()} ({SAMPLES_DF['has_tests'].mean()*100:.0f}%)",
    }

    for k, v in stats.items():
        print(f"  {k}: {v}")

    print("\nSource distribution:")
    for source, count in SAMPLES_DF['source'].value_counts().head(5).items():
        print(f"  {source}: {count}")
else:
    print("  ❌ No samples loaded - check dataset structure above")

# ============================================================================
# SAMPLE PREVIEW
# ============================================================================

print("\n" + "-" * 60)
print("Sample Preview (first sample):")
print("-" * 60)

if SAMPLES:
    sample = SAMPLES[0]
    print(f"\nID: {sample.id}")
    print(f"Source: {sample.source}")
    print(f"\nProblem (first 200 chars):\n  {sample.problem[:200]}...")
    print(f"\nThinking (first 300 chars):\n  {sample.thinking[:300]}...")
    print(f"\nSolution (first 200 chars):\n  {sample.solution[:200]}...")
else:
    print("\n  No samples to preview")

# ============================================================================
# EXPORTS
# ============================================================================

@dataclass
class DatasetExports:
    """Exports from Cell 3."""
    samples: List[CodeSample]
    df: pd.DataFrame
    n_samples: int

    def get_sample(self, idx: int) -> CodeSample:
        return self.samples[idx]

    def get_by_id(self, sample_id: str) -> Optional[CodeSample]:
        for s in self.samples:
            if s.id == sample_id:
                return s
        return None

DATASET = DatasetExports(
    samples=SAMPLES,
    df=SAMPLES_DF,
    n_samples=len(SAMPLES),
)

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "=" * 60)
print("✅ CELL 3 COMPLETE")
print("=" * 60)
print(f"""
Exports:
  ├── DATASET.samples: List[CodeSample] ({len(SAMPLES)} items)
  ├── DATASET.df: DataFrame with metadata
  ├── DATASET.get_sample(idx): Get by index
  └── DATASET.get_by_id(id): Get by ID

CodeSample fields:
  ├── .id, .problem, .thinking, .solution, .source
  ├── .test_cases (if available)
  └── .is_valid, .has_thinking, .has_solution

Proceed to Cell 4: CoT Parser
""")

In [ ]:
"""
CELL 3: LOAD OPENTHOUGHTS DATASET
=================================
Load DeepSeek-R1 reasoning traces for code generation problems.
"""

import re
import pandas as pd
from datasets import load_dataset
from dataclasses import dataclass, field
from typing import List, Set, Optional, Tuple

print("=" * 60)
print("CELL 3: Load OpenThoughts Dataset")
print("=" * 60)

# ============================================================================
# LOAD DATASET
# ============================================================================

print("\n[1/4] Loading OpenThoughts-114k...")

ds = load_dataset("open-thoughts/OpenThoughts-114k", split="train")
print(f"  ✅ Loaded {len(ds):,} total samples")

# Inspect structure
print(f"\n  Dataset columns: {ds.column_names}")

# This dataset uses conversations format!
# conversations = [{'from': 'user', 'value': '...'}, {'from': 'assistant', 'value': '...'}]
sample_ex = ds[0]
print(f"  Format: conversations list")
if 'conversations' in sample_ex:
    convs = sample_ex['conversations']
    print(f"  Conversation turns: {len(convs)}")
    for i, turn in enumerate(convs[:3]):
        role = turn.get('from', 'unknown')
        val = str(turn.get('value', ''))[:100]
        print(f"    [{i}] {role}: {val}...")

# ============================================================================
# PARSE CONVERSATIONS FORMAT
# ============================================================================

def parse_conversation(example: dict) -> dict:
    """Extract problem, reasoning, solution from conversations format."""
    convs = example.get('conversations', [])

    problem = ""
    reasoning = ""
    solution = ""

    for turn in convs:
        role = turn.get('from', '')
        value = turn.get('value', '')

        if role == 'user':
            problem = value
        elif role == 'assistant':
            # Assistant response contains both reasoning and solution
            # Split on common patterns
            full_response = value

            # Extract <think>...</think> or similar
            think_match = re.search(r'<think>(.*?)</think>', full_response, re.DOTALL)
            if think_match:
                reasoning = think_match.group(1).strip()
                # Solution is everything after </think>
                solution = full_response[think_match.end():].strip()
            else:
                # Try to split on code block
                code_match = re.search(r'```(?:python)?\s*(.*?)```', full_response, re.DOTALL)
                if code_match:
                    solution = code_match.group(1).strip()
                    # Everything before the code block is reasoning
                    reasoning = full_response[:code_match.start()].strip()
                else:
                    # Just use the whole thing as solution
                    solution = full_response

    return {
        'problem': problem,
        'reasoning': reasoning,
        'solution': solution,
    }

# ============================================================================
# FILTER FOR CODE DOMAIN
# ============================================================================

print("\n[2/4] Filtering for code problems...")

def is_code_sample(example: dict) -> bool:
    """Check if sample contains Python code."""
    parsed = parse_conversation(example)
    solution = parsed['solution']

    # Check for Python code indicators
    code_indicators = ['def ', 'class ', 'import ', 'return ', 'for ', 'while ']
    return any(ind in solution for ind in code_indicators)

# Filter with progress
print("  Scanning for code samples...")
code_samples = []
for i, ex in enumerate(ds):
    if is_code_sample(ex):
        code_samples.append(ex)
    if i % 20000 == 0:
        print(f"    Scanned {i:,}... found {len(code_samples):,} code samples")
    # Early stop for testing - remove this line for full dataset
    # if len(code_samples) >= 500: break

print(f"  ✅ Found {len(code_samples):,} code samples")

# ============================================================================
# PARSE AND VALIDATE SAMPLES
# ============================================================================

print("\n[3/4] Parsing samples...")

@dataclass
class CodeSample:
    """A parsed code sample with reasoning trace."""
    id: str
    problem: str
    thinking: str
    solution: str
    source: str
    test_cases: Optional[str] = None

    @property
    def has_thinking(self) -> bool:
        return len(self.thinking) > 50

    @property
    def has_solution(self) -> bool:
        return 'def ' in self.solution or 'class ' in self.solution

    @property
    def is_valid(self) -> bool:
        return self.has_thinking and self.has_solution

def extract_code(text: str) -> str:
    """Extract Python code from solution."""
    # Try ```python block
    match = re.search(r'```python\s*(.*?)```', text, re.DOTALL)
    if match:
        return match.group(1).strip()

    # Try any ``` block
    match = re.search(r'```\s*(.*?)```', text, re.DOTALL)
    if match:
        code = match.group(1).strip()
        if 'def ' in code or 'class ' in code:
            return code

    # Return raw text if it looks like code
    if 'def ' in text or 'class ' in text:
        return text.strip()

    return ""

def extract_test_cases(problem: str, solution: str) -> Optional[str]:
    """Try to extract test cases from problem or solution."""
    asserts = re.findall(r'assert\s+.+', solution)
    if asserts:
        return '\n'.join(asserts[:5])
    return None

def parse_sample(idx: int, example: dict) -> CodeSample:
    """Parse a raw example into CodeSample."""
    parsed = parse_conversation(example)

    problem = parsed['problem']
    reasoning = parsed['reasoning']
    solution = parsed['solution']

    # Clean up solution - extract just the code
    code = extract_code(solution) if solution else solution
    if not code:
        code = solution  # Use raw if extraction fails

    return CodeSample(
        id=f"ot_{idx:04d}",
        problem=problem,
        thinking=reasoning,
        solution=code,
        source='OpenThoughts',
        test_cases=extract_test_cases(problem, code) if problem and code else None,
    )

# Parse all code samples
parsed_samples = [parse_sample(i, ex) for i, ex in enumerate(code_samples)]
valid_samples = [s for s in parsed_samples if s.is_valid]

print(f"  ✅ Parsed {len(parsed_samples):,} samples")
print(f"  ✅ Valid (has thinking + code): {len(valid_samples):,}")

# If no valid samples, show debug info
if len(valid_samples) == 0 and len(parsed_samples) > 0:
    print("\n  ⚠️  No valid samples! Debugging first parsed sample:")
    s = parsed_samples[0]
    print(f"    thinking length: {len(s.thinking)}")
    print(f"    solution length: {len(s.solution)}")
    print(f"    has_thinking: {s.has_thinking}")
    print(f"    has_solution: {s.has_solution}")
    if s.thinking:
        print(f"    thinking preview: {s.thinking[:200]}...")
    if s.solution:
        print(f"    solution preview: {s.solution[:200]}...")

# ============================================================================
# SELECT FINAL SAMPLE
# ============================================================================

print("\n[4/4] Selecting samples...")

import random
random.seed(ENV_CONFIG.RANDOM_SEED)

if len(valid_samples) == 0:
    print("  ❌ No valid samples found!")
    print("\n  Falling back: using samples with ANY code (relaxed validation)")
    # Relax validation - just need some code
    valid_samples = [s for s in parsed_samples if len(s.solution) > 20]
    print(f"  ✅ Relaxed: {len(valid_samples)} samples with code")

if len(valid_samples) == 0:
    print("  ❌ Still no samples! Using raw examples directly...")
    # Last resort: parse from raw conversations
    for i, ex in enumerate(code_samples[:200]):
        parsed = parse_conversation(ex)
        if 'def ' in parsed['solution']:
            valid_samples.append(CodeSample(
                id=f"ot_{i:04d}",
                problem=parsed['problem'][:1000],
                thinking=parsed['reasoning'],
                solution=parsed['solution'],
                source='OpenThoughts',
            ))
    print(f"  ✅ Direct extraction: {len(valid_samples)} samples")

N_SAMPLES = min(ENV_CONFIG.N_TOTAL, len(valid_samples))
SAMPLES = random.sample(valid_samples, N_SAMPLES) if valid_samples else []

# Sort by ID for reproducibility
SAMPLES.sort(key=lambda x: x.id)

print(f"  ✅ Selected {N_SAMPLES} samples")

# ============================================================================
# CREATE DATAFRAME
# ============================================================================

if SAMPLES:
    SAMPLES_DF = pd.DataFrame([
        {
            'id': s.id,
            'problem': s.problem[:500],
            'thinking': s.thinking,
            'solution': s.solution,
            'source': s.source,
            'thinking_len': len(s.thinking),
            'solution_len': len(s.solution),
            'has_tests': s.test_cases is not None,
        }
        for s in SAMPLES
    ])
else:
    SAMPLES_DF = pd.DataFrame(columns=['id', 'problem', 'thinking', 'solution',
                                        'source', 'thinking_len', 'solution_len', 'has_tests'])

# ============================================================================
# STATISTICS
# ============================================================================

print("\n" + "-" * 60)
print("Dataset Statistics:")
print("-" * 60)

if len(SAMPLES) > 0:
    stats = {
        'Total samples': len(SAMPLES),
        'Avg thinking length': f"{SAMPLES_DF['thinking_len'].mean():.0f} chars",
        'Avg solution length': f"{SAMPLES_DF['solution_len'].mean():.0f} chars",
        'With test cases': f"{SAMPLES_DF['has_tests'].sum()} ({SAMPLES_DF['has_tests'].mean()*100:.0f}%)",
    }

    for k, v in stats.items():
        print(f"  {k}: {v}")

    print("\nSource distribution:")
    for source, count in SAMPLES_DF['source'].value_counts().head(5).items():
        print(f"  {source}: {count}")
else:
    print("  ❌ No samples loaded - check dataset structure above")

# ============================================================================
# SAMPLE PREVIEW
# ============================================================================

print("\n" + "-" * 60)
print("Sample Preview (first sample):")
print("-" * 60)

if SAMPLES:
    sample = SAMPLES[0]
    print(f"\nID: {sample.id}")
    print(f"Source: {sample.source}")
    print(f"\nProblem (first 200 chars):\n  {sample.problem[:200]}...")
    print(f"\nThinking (first 300 chars):\n  {sample.thinking[:300]}...")
    print(f"\nSolution (first 200 chars):\n  {sample.solution[:200]}...")
else:
    print("\n  No samples to preview")

# ============================================================================
# EXPORTS
# ============================================================================

@dataclass
class DatasetExports:
    """Exports from Cell 3."""
    samples: List[CodeSample]
    df: pd.DataFrame
    n_samples: int

    def get_sample(self, idx: int) -> CodeSample:
        return self.samples[idx]

    def get_by_id(self, sample_id: str) -> Optional[CodeSample]:
        for s in self.samples:
            if s.id == sample_id:
                return s
        return None

DATASET = DatasetExports(
    samples=SAMPLES,
    df=SAMPLES_DF,
    n_samples=len(SAMPLES),
)

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "=" * 60)
print("✅ CELL 3 COMPLETE")
print("=" * 60)
print(f"""
Exports:
  ├── DATASET.samples: List[CodeSample] ({len(SAMPLES)} items)
  ├── DATASET.df: DataFrame with metadata
  ├── DATASET.get_sample(idx): Get by index
  └── DATASET.get_by_id(id): Get by ID

CodeSample fields:
  ├── .id, .problem, .thinking, .solution, .source
  ├── .test_cases (if available)
  └── .is_valid, .has_thinking, .has_solution

Proceed to Cell 4: CoT Parser
""")

In [ ]:
"""
CELL 4: COT SEGMENTER
=====================
Split raw thinking into individual reasoning segments for DFA analysis.
"""

import re
from dataclasses import dataclass, field
from typing import List, Set, Tuple

print("=" * 60)
print("CELL 4: CoT Segmenter")
print("=" * 60)

# ============================================================================
# DATA STRUCTURES
# ============================================================================

@dataclass
class Segment:
    """A single reasoning step from CoT."""
    id: str
    text: str
    position: int
    concepts: Set[str] = field(default_factory=set)

    def __repr__(self):
        return f"Segment({self.id}, {len(self.text)} chars, {len(self.concepts)} concepts)"

# ============================================================================
# SEGMENTATION FUNCTIONS
# ============================================================================

def clean_thinking(text: str) -> str:
    """Remove thinking tags and clean whitespace."""
    # Remove various thinking tags
    patterns = [
        r'<\|begin_of_thought\|>',
        r'<\|end_of_thought\|>',
        r'<think>',
        r'</think>',
        r'<reasoning>',
        r'</reasoning>',
    ]
    for p in patterns:
        text = re.sub(p, '', text, flags=re.IGNORECASE)
    return text.strip()

def split_into_sentences(text: str) -> List[str]:
    """Split text into sentences, handling code blocks."""
    # Protect code blocks
    code_blocks = []
    def save_code(m):
        code_blocks.append(m.group(0))
        return f"__CODE_BLOCK_{len(code_blocks)-1}__"

    text = re.sub(r'```.*?```', save_code, text, flags=re.DOTALL)
    text = re.sub(r'`[^`]+`', save_code, text)

    # Split on sentence boundaries
    # Handle: . ! ? followed by space and capital, or newline
    sentences = re.split(r'(?<=[.!?])\s+(?=[A-Z])|(?<=\n)\s*(?=\w)', text)

    # Restore code blocks
    result = []
    for s in sentences:
        for i, block in enumerate(code_blocks):
            s = s.replace(f"__CODE_BLOCK_{i}__", block)
        s = s.strip()
        if s:
            result.append(s)

    return result

def split_by_markers(text: str) -> List[str]:
    """Split by explicit step markers (1. 2. 3. or - or *)."""
    # Try numbered steps: "1." "2." etc
    numbered = re.split(r'\n\s*\d+[.)]\s+', text)
    if len(numbered) > 2:
        return [s.strip() for s in numbered if s.strip()]

    # Try bullet points
    bullets = re.split(r'\n\s*[-*•]\s+', text)
    if len(bullets) > 2:
        return [s.strip() for s in bullets if s.strip()]

    # Try paragraph breaks (double newline)
    paragraphs = re.split(r'\n\s*\n', text)
    if len(paragraphs) > 1:
        return [s.strip() for s in paragraphs if s.strip()]

    return []

def segment_cot(thinking: str, min_length: int = 20) -> List[Segment]:
    """
    Segment CoT into reasoning steps.

    Strategy:
    1. Try explicit markers (1. 2. 3. or bullets)
    2. Fall back to sentence splitting
    3. Merge very short segments
    """
    cleaned = clean_thinking(thinking)

    if not cleaned:
        return []

    # Try marker-based splitting first
    parts = split_by_markers(cleaned)

    # Fall back to sentence splitting
    if len(parts) < 3:
        parts = split_into_sentences(cleaned)

    # Filter short segments and create Segment objects
    segments = []
    for i, text in enumerate(parts):
        if len(text) >= min_length:
            segments.append(Segment(
                id=f"s{i}",
                text=text,
                position=i,
                concepts=set(),  # Filled in Cell 5
            ))

    # Merge consecutive short segments if we have too few
    if len(segments) < 3 and len(cleaned) > 200:
        # Just chunk by ~200 chars
        chunks = [cleaned[i:i+200] for i in range(0, len(cleaned), 200)]
        segments = [
            Segment(id=f"s{i}", text=chunk.strip(), position=i, concepts=set())
            for i, chunk in enumerate(chunks) if len(chunk.strip()) >= min_length
        ]

    return segments

# ============================================================================
# BATCH PROCESSING
# ============================================================================

def segment_all_samples(samples: list) -> dict:
    """Segment all samples, return dict mapping id -> segments."""
    results = {}
    stats = {'total': 0, 'min': float('inf'), 'max': 0, 'sum': 0}

    for sample in samples:
        segments = segment_cot(sample.thinking)
        results[sample.id] = segments

        n = len(segments)
        stats['total'] += 1
        stats['sum'] += n
        stats['min'] = min(stats['min'], n)
        stats['max'] = max(stats['max'], n)

    stats['avg'] = stats['sum'] / stats['total'] if stats['total'] > 0 else 0
    return results, stats

# ============================================================================
# PROCESS DATASET
# ============================================================================

print("\n[1/2] Segmenting CoT traces...")

SEGMENTED, seg_stats = segment_all_samples(DATASET.samples)

print(f"  ✅ Segmented {seg_stats['total']} samples")
print(f"  ✅ Segments per sample: min={seg_stats['min']}, avg={seg_stats['avg']:.1f}, max={seg_stats['max']}")

# ============================================================================
# PREVIEW
# ============================================================================

print("\n[2/2] Preview...")
print("-" * 60)

# Show first sample's segments
sample = DATASET.samples[0]
segments = SEGMENTED[sample.id]

print(f"Sample: {sample.id}")
print(f"Total segments: {len(segments)}")
print(f"\nFirst 3 segments:")

for seg in segments[:3]:
    preview = seg.text[:100].replace('\n', ' ')
    print(f"\n  [{seg.id}] ({len(seg.text)} chars)")
    print(f"      {preview}...")

# ============================================================================
# EXPORTS
# ============================================================================

@dataclass
class SegmenterExports:
    """Exports from Cell 4."""
    segmented: dict  # sample_id -> List[Segment]
    segment_cot: callable
    stats: dict

SEGMENTER = SegmenterExports(
    segmented=SEGMENTED,
    segment_cot=segment_cot,
    stats=seg_stats,
)

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "=" * 60)
print("✅ CELL 4 COMPLETE")
print("=" * 60)
print(f"""
Stats:
  ├── Samples processed: {seg_stats['total']}
  ├── Avg segments/sample: {seg_stats['avg']:.1f}
  ├── Total segments: {seg_stats['sum']}
  └── Segment length filter: ≥20 chars

Exports:
  ├── SEGMENTER.segmented[sample_id] → List[Segment]
  ├── SEGMENTER.segment_cot(text) → List[Segment]
  └── SEGMENTER.stats

Segment fields:
  ├── .id ("s0", "s1", ...)
  ├── .text (raw reasoning text)
  ├── .position (order in CoT)
  └── .concepts (empty, filled in Cell 5)

Proceed to Cell 5: Concept Extraction
""")

In [ ]:
"""
CELL 5: CONCEPT EXTRACTION
==========================
Extract programming concepts from CoT segments and code for reaching definitions.
"""

import re
import ast
from dataclasses import dataclass, field
from typing import List, Set, Dict, Tuple, Optional

print("=" * 60)
print("CELL 5: Concept Extraction")
print("=" * 60)

# ============================================================================
# CONCEPT VOCABULARY (22 PROGRAMMING CONCEPTS)
# ============================================================================

CONCEPT_VOCABULARY: Dict[str, Set[str]] = {
    # Data Structures (8)
    'dict': {'hash', 'map', 'dictionary', 'hashmap', 'hash map', 'key-value',
             'lookup table', 'mapping', 'counter', 'dict', '{}'},
    'list': {'array', 'list', 'sequence', 'collection', 'elements', 'items', '[]'},
    'set': {'set', 'unique', 'deduplicate', 'distinct', 'set()'},
    'stack': {'stack', 'lifo', 'push', 'pop', 'append and pop'},
    'queue': {'queue', 'fifo', 'deque', 'bfs', 'collections.deque'},
    'heap': {'heap', 'priority queue', 'heapq', 'heappush', 'heappop', 'min heap', 'max heap'},
    'tree': {'tree', 'binary tree', 'bst', 'trie', 'node', 'root', 'left child', 'right child'},
    'graph': {'graph', 'vertices', 'edges', 'adjacent', 'neighbor', 'dfs', 'adjacency'},

    # Algorithms (7)
    'sort': {'sort', 'order', 'arrange', 'sorted', 'ascending', 'descending', 'sorted()'},
    'search': {'search', 'find', 'lookup', 'binary search', 'locate', 'bisect'},
    'recursion': {'recursive', 'recursion', 'base case', 'call itself', 'recur'},
    'dp': {'dynamic programming', 'memoization', 'memo', 'dp', 'subproblem', 'cache', 'lru_cache'},
    'greedy': {'greedy', 'local optimal', 'best choice', 'optimal substructure'},
    'two_pointer': {'two pointer', 'left right', 'start end', 'sliding window', 'window'},
    'backtrack': {'backtrack', 'prune', 'explore', 'candidates', 'backtracking'},

    # Control Flow (3)
    'loop': {'iterate', 'loop', 'for each', 'traverse', 'go through', 'while', 'for'},
    'condition': {'if', 'check', 'condition', 'edge case', 'boundary', 'elif', 'else'},
    'early_return': {'return early', 'base case', 'edge case', 'special case', 'return'},

    # Operations (4)
    'count': {'count', 'frequency', 'occurrences', 'how many', 'counter'},
    'sum': {'sum', 'total', 'add up', 'accumulate', 'sum()'},
    'max_min': {'maximum', 'minimum', 'max', 'min', 'largest', 'smallest', 'max()', 'min()'},
    'string_op': {'string', 'character', 'substring', 'split', 'join', 'strip', 'str'},
}

# Reverse mapping: keyword -> concept
KEYWORD_TO_CONCEPT: Dict[str, str] = {}
for concept, keywords in CONCEPT_VOCABULARY.items():
    for kw in keywords:
        KEYWORD_TO_CONCEPT[kw.lower()] = concept

# ============================================================================
# COT CONCEPT EXTRACTION
# ============================================================================

def extract_cot_concepts(text: str) -> Set[str]:
    """Extract programming concepts from CoT text."""
    text_lower = text.lower()
    found = set()

    # Check each keyword
    for keyword, concept in KEYWORD_TO_CONCEPT.items():
        # Use word boundary for short keywords to avoid false positives
        if len(keyword) <= 3:
            if re.search(rf'\b{re.escape(keyword)}\b', text_lower):
                found.add(concept)
        else:
            if keyword in text_lower:
                found.add(concept)

    return found

# ============================================================================
# AST NODE TO CONCEPT MAPPING
# ============================================================================

AST_TO_CONCEPT: Dict[str, str] = {
    # Data structures
    'Dict': 'dict',
    'DictComp': 'dict',
    'List': 'list',
    'ListComp': 'list',
    'Set': 'set',
    'SetComp': 'set',
    'Tuple': 'list',  # Treat tuple as list-like

    # Control flow
    'For': 'loop',
    'While': 'loop',
    'AsyncFor': 'loop',
    'If': 'condition',
    'IfExp': 'condition',
    'Return': 'early_return',

    # Comprehensions indicate loops
    'comprehension': 'loop',
}

# Function calls to concepts
CALL_TO_CONCEPT: Dict[str, str] = {
    # Built-ins
    'sorted': 'sort',
    'sort': 'sort',
    'max': 'max_min',
    'min': 'max_min',
    'sum': 'sum',
    'len': 'count',
    'count': 'count',
    'range': 'loop',
    'enumerate': 'loop',
    'zip': 'loop',
    'map': 'loop',
    'filter': 'loop',

    # Collections
    'dict': 'dict',
    'list': 'list',
    'set': 'set',
    'deque': 'queue',
    'Counter': 'dict',
    'defaultdict': 'dict',
    'OrderedDict': 'dict',

    # Heap
    'heappush': 'heap',
    'heappop': 'heap',
    'heapify': 'heap',
    'heapreplace': 'heap',

    # Search
    'bisect': 'search',
    'bisect_left': 'search',
    'bisect_right': 'search',
    'index': 'search',
    'find': 'search',

    # String
    'split': 'string_op',
    'join': 'string_op',
    'strip': 'string_op',
    'replace': 'string_op',
    'lower': 'string_op',
    'upper': 'string_op',

    # DP/Memoization
    'lru_cache': 'dp',
    'cache': 'dp',
}

# ============================================================================
# CODE ELEMENT DATA STRUCTURE
# ============================================================================

@dataclass
class CodeElement:
    """A code construct that may be linked to CoT reasoning."""
    id: str
    node_type: str
    line_number: int
    concepts: Set[str] = field(default_factory=set)
    source_text: str = ""

    def __repr__(self):
        return f"CodeElement({self.id}, {self.node_type}, line {self.line_number}, {self.concepts})"

# ============================================================================
# AST CONCEPT EXTRACTION
# ============================================================================

class ConceptVisitor(ast.NodeVisitor):
    """AST visitor that extracts concepts from code."""

    def __init__(self, source_lines: List[str]):
        self.elements: List[CodeElement] = []
        self.concepts: Set[str] = set()
        self.source_lines = source_lines
        self.element_counter = 0

    def _add_element(self, node: ast.AST, node_type: str, concepts: Set[str]):
        """Create and store a CodeElement."""
        line = getattr(node, 'lineno', 0)
        source = self.source_lines[line-1].strip() if 0 < line <= len(self.source_lines) else ""

        elem = CodeElement(
            id=f"c{self.element_counter}",
            node_type=node_type,
            line_number=line,
            concepts=concepts,
            source_text=source[:100],
        )
        self.elements.append(elem)
        self.concepts.update(concepts)
        self.element_counter += 1

    def visit_Dict(self, node):
        self._add_element(node, 'Dict', {'dict'})
        self.generic_visit(node)

    def visit_DictComp(self, node):
        self._add_element(node, 'DictComp', {'dict', 'loop'})
        self.generic_visit(node)

    def visit_List(self, node):
        self._add_element(node, 'List', {'list'})
        self.generic_visit(node)

    def visit_ListComp(self, node):
        self._add_element(node, 'ListComp', {'list', 'loop'})
        self.generic_visit(node)

    def visit_Set(self, node):
        self._add_element(node, 'Set', {'set'})
        self.generic_visit(node)

    def visit_SetComp(self, node):
        self._add_element(node, 'SetComp', {'set', 'loop'})
        self.generic_visit(node)

    def visit_For(self, node):
        self._add_element(node, 'For', {'loop'})
        self.generic_visit(node)

    def visit_While(self, node):
        self._add_element(node, 'While', {'loop'})
        self.generic_visit(node)

    def visit_If(self, node):
        self._add_element(node, 'If', {'condition'})
        self.generic_visit(node)

    def visit_Return(self, node):
        self._add_element(node, 'Return', {'early_return'})
        self.generic_visit(node)

    def visit_Call(self, node):
        """Handle function calls."""
        func_name = None

        # Get function name
        if isinstance(node.func, ast.Name):
            func_name = node.func.id
        elif isinstance(node.func, ast.Attribute):
            func_name = node.func.attr

        if func_name and func_name in CALL_TO_CONCEPT:
            concept = CALL_TO_CONCEPT[func_name]
            self._add_element(node, f'Call:{func_name}', {concept})

        self.generic_visit(node)

    def visit_FunctionDef(self, node):
        """Check for recursive calls."""
        # Look for self-calls (recursion)
        for child in ast.walk(node):
            if isinstance(child, ast.Call):
                if isinstance(child.func, ast.Name) and child.func.id == node.name:
                    self._add_element(node, 'Recursion', {'recursion'})
                    break
        self.generic_visit(node)

def extract_code_concepts(code: str) -> Tuple[Set[str], List[CodeElement]]:
    """
    Extract concepts from Python code using AST analysis.

    Returns:
        (all_concepts, list_of_code_elements)
    """
    try:
        tree = ast.parse(code)
    except SyntaxError:
        return set(), []

    source_lines = code.split('\n')
    visitor = ConceptVisitor(source_lines)
    visitor.visit(tree)

    return visitor.concepts, visitor.elements

# ============================================================================
# UPDATE SEGMENTS WITH CONCEPTS
# ============================================================================

def enrich_segments(segments: List[Segment]) -> List[Segment]:
    """Add concepts to each segment."""
    for seg in segments:
        seg.concepts = extract_cot_concepts(seg.text)
    return segments

# ============================================================================
# TESTS
# ============================================================================

def run_tests():
    """Validate concept extraction."""
    print("\n[TESTS] Running validation...")
    results = []

    # Test 1: CoT hash map/sort
    c = extract_cot_concepts("I'll use a hash map for O(1) lookup, then sort the results")
    results.append(('dict' in c and 'sort' in c, "CoT: hash map→dict, sort→sort"))

    # Test 2: CoT loop/array
    c = extract_cot_concepts("I'll iterate through each element in the array")
    results.append(('loop' in c and 'list' in c, "CoT: iterate→loop, array→list"))

    # Test 3: AST dict
    c, _ = extract_code_concepts("seen = {}")
    results.append(('dict' in c, "AST: {}→dict"))

    # Test 4: AST for loop
    c, _ = extract_code_concepts("for i in range(10):\n    print(i)")
    results.append(('loop' in c, "AST: for→loop"))

    # Test 5: AST sorted
    c, _ = extract_code_concepts("result = sorted(nums)")
    results.append(('sort' in c, "AST: sorted()→sort"))

    # Test 6: AST heap
    c, _ = extract_code_concepts("import heapq\nheapq.heappush(h, item)")
    results.append(('heap' in c, "AST: heappush→heap"))

    # Test 7: AST recursion
    c, _ = extract_code_concepts("def fib(n):\n    if n <= 1: return n\n    return fib(n-1) + fib(n-2)")
    results.append(('recursion' in c, "AST: self-call→recursion"))

    # Test 8: Multiple concepts
    code = "def solve(nums):\n    seen = {}\n    for n in nums:\n        if n in seen: return True\n        seen[n] = True\n    return False"
    c, _ = extract_code_concepts(code)
    results.append(({'dict','loop','condition','early_return'}.issubset(c), "AST: multi-concept"))

    # Test 9: Invalid code
    c, e = extract_code_concepts("this is not valid python {{{{")
    results.append((c == set() and e == [], "Invalid code→empty"))

    # Test 10: DP in CoT
    c = extract_cot_concepts("use dynamic programming with memoization to cache results")
    results.append(('dp' in c, "CoT: DP/memoization→dp"))

    passed = sum(1 for r, _ in results if r)
    for ok, desc in results:
        print(f"  {'✅' if ok else '❌'} {desc}")
    print(f"\n  Results: {passed}/{len(results)} tests passed")
    return passed, len(results) - passed

# Run tests
test_passed, test_failed = run_tests()

# ============================================================================
# PROCESS DATASET
# ============================================================================

print("\n" + "-" * 60)
print("[1/3] Extracting concepts from CoT segments...")

cot_concept_stats = {'total_segments': 0, 'with_concepts': 0, 'concept_counts': {}}

for sample_id, segments in SEGMENTER.segmented.items():
    enriched = enrich_segments(segments)
    SEGMENTER.segmented[sample_id] = enriched

    for seg in enriched:
        cot_concept_stats['total_segments'] += 1
        if seg.concepts:
            cot_concept_stats['with_concepts'] += 1
        for c in seg.concepts:
            cot_concept_stats['concept_counts'][c] = cot_concept_stats['concept_counts'].get(c, 0) + 1

print(f"  ✅ Processed {cot_concept_stats['total_segments']} segments")
print(f"  ✅ Segments with concepts: {cot_concept_stats['with_concepts']} ({100*cot_concept_stats['with_concepts']/max(1,cot_concept_stats['total_segments']):.0f}%)")

# ============================================================================
# EXTRACT CODE CONCEPTS
# ============================================================================

print("\n[2/3] Extracting concepts from code...")

CODE_ANALYSIS: Dict[str, Tuple[Set[str], List[CodeElement]]] = {}
code_concept_stats = {'total_elements': 0, 'concept_counts': {}}

for sample in DATASET.samples:
    concepts, elements = extract_code_concepts(sample.solution)
    CODE_ANALYSIS[sample.id] = (concepts, elements)

    code_concept_stats['total_elements'] += len(elements)
    for c in concepts:
        code_concept_stats['concept_counts'][c] = code_concept_stats['concept_counts'].get(c, 0) + 1

print(f"  ✅ Processed {len(CODE_ANALYSIS)} samples")
print(f"  ✅ Total code elements: {code_concept_stats['total_elements']}")

# ============================================================================
# CONCEPT DISTRIBUTION
# ============================================================================

print("\n[3/3] Concept distribution...")

top_cot = sorted(cot_concept_stats['concept_counts'].items(), key=lambda x: -x[1])[:5]
top_code = sorted(code_concept_stats['concept_counts'].items(), key=lambda x: -x[1])[:5]

print(f"  Top CoT: {', '.join(f'{c}:{n}' for c,n in top_cot)}")
print(f"  Top Code: {', '.join(f'{c}:{n}' for c,n in top_code)}")

# ============================================================================
# PREVIEW
# ============================================================================

print("\n" + "-" * 60)
sample = DATASET.samples[0]
segments = SEGMENTER.segmented[sample.id]
code_concepts, code_elements = CODE_ANALYSIS[sample.id]

print(f"Preview ({sample.id}): {len(segments)} segs, {len(code_elements)} code elems")
print(f"  CoT concepts: {[s.concepts for s in segments[:3] if s.concepts]}")
print(f"  Code concepts: {code_concepts}")

# ============================================================================
# EXPORTS
# ============================================================================

@dataclass
class ConceptExports:
    """Exports from Cell 5."""
    vocabulary: Dict[str, Set[str]]
    extract_cot: callable
    extract_code: callable
    code_analysis: Dict[str, Tuple[Set[str], List[CodeElement]]]
    cot_stats: dict
    code_stats: dict

CONCEPTS = ConceptExports(
    vocabulary=CONCEPT_VOCABULARY,
    extract_cot=extract_cot_concepts,
    extract_code=extract_code_concepts,
    code_analysis=CODE_ANALYSIS,
    cot_stats=cot_concept_stats,
    code_stats=code_concept_stats,
)

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "=" * 60)
print("✅ CELL 5 COMPLETE")
print("=" * 60)
print(f"""
Tests: {test_passed}/{test_passed+test_failed} passed

Stats:
  ├── CoT segments: {cot_concept_stats['total_segments']}
  ├── Segments with concepts: {cot_concept_stats['with_concepts']}
  ├── Code elements: {code_concept_stats['total_elements']}
  └── Concept vocabulary: {len(CONCEPT_VOCABULARY)} concepts

Exports:
  ├── CONCEPTS.vocabulary → Dict[concept, keywords]
  ├── CONCEPTS.extract_cot(text) → Set[str]
  ├── CONCEPTS.extract_code(code) → (Set, List[CodeElement])
  ├── CONCEPTS.code_analysis[sample_id] → (concepts, elements)
  └── SEGMENTER.segmented[id] now has concepts filled

Proceed to Cell 6: Reaching Definitions
""")